# Hidden Tests

In this file, the hidden tests for all the rubric points are to be described. The tests for the individual rubric points are enclosed within `# BEGIN <rubric_point>` and `# END <rubric_point>` NBConvert cells. `hidden_tests.py` works by executing the contents of those cells between those two tags for each `<rubric_point>`. In order to initialize variables, `hidden_tests.py` also executes all code within `BEGIN` and `END` tags that appear before the `original` test.

Code that is not enclosed within `BEGIN` and `END` tags are not executed by `hidden_tests.py`. They are used for generating the hidden datasets.

In [1]:
from hidden_tests import *
import otter_tests.gen_public_tests as gen_public_tests
import os, csv, json, copy, shutil
import random
import numpy as np

In [2]:
DIRECTORY = '..'
FILE = 'p11.ipynb'

In [3]:
results = {}

In [4]:
deductions = {}
rubric = parse_rubric_file(os.path.join(DIRECTORY, "rubric.md"))
directories = get_directories(rubric)
comments = get_all_comments(directories)

In [5]:
def write_readme(data, write_path):
    """write_readme(data, write_path) writes the contents of `data` into the README.txt file `write_path`"""
    f = open(write_path, encoding='utf-8')
    rubric_point = f.read().split("\n")[0].strip(" \n")
    f.close()
    
    f = open(write_path, 'w', encoding='utf-8')
    f.write(rubric_point + "\n\n" + data)
    f.close()

In [6]:
def copy_over(src, trgt):
    """copy_over(src, trgt) copies the contents of `src` over to `trgt` if `src` is a file or a directory"""
    if os.path.isfile(src):
        if os.path.exists(trgt):
            os.remove(trgt)
        shutil.copy(src, trgt)
    elif os.path.isdir(src):
        if os.path.exists(trgt):
            shutil.rmtree(trgt)
        shutil.copy_tree(src, trgt)

## Variables

Useful variables that are used by many rubric tests can be stored here. The contents of this tag will be executed before each rubric test, so these variables get initialized before each rubric test.

`verify_fn_defn` defines the function `verify_fn` which is used for verifying if the function `expected` and `actual` have the same outputs for all permutations of inputs from `var_lists`.

In [7]:
verify_fn_defn = """
def verify_fn(expected, actual, var_inputs, test_format):
    for var in var_inputs:
        try:
            actual_val = actual(*var)
        except Exception as e:
            output = "%s results: " % actual.__name__
            output += "%s error enountered on %s%s" % (type(e).__name__, actual.__name__, repr(var))
            return output
        expected_val = expected(*var)
        check = public_tests.compare(expected_val, actual_val, test_format)
        if check != public_tests.PASS:
            output = "%s results: " % actual.__name__
            output += "%s%s output: %s" % (actual.__name__, repr(var), check)
            return output
    return "%s results: All test cases passed!" % actual.__name__"""

`function_dependencies_functions` stores the previously defined functions that each function definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required function is used. For these rubric points, when we test a particular function, we use `function_dependencies_functions` to ensure that all the functions that it depends on are replaced with logically correct versions. This helps isolate the issue with the functions.

In [8]:
function_dependencies_functions = {}
function_dependencies_functions['star_cell'] = []
function_dependencies_functions['get_stars'] = ['star_cell']
function_dependencies_functions['planet_cell'] = []
function_dependencies_functions['get_planets'] = ['planet_cell']
function_dependencies_functions['get_all_paths_in'] = []
function_dependencies_functions['get_surface_gravity'] = ['get_planets', 'planet_cell']
function_dependencies_functions['get_distances_to_star'] = ['get_planets', 'planet_cell']
function_dependencies_functions['get_liquid_water_distances'] = ['get_stars', 'star_cell', 'get_planets', 'planet_cell']
function_dependencies_functions['get_surface_temperatures'] = ['get_planets', 'planet_cell']

`function_dependencies_data_structures` stores the previously defined data structures that each function definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required function is used. For these rubric points, when we test a particular function, we use `function_dependencies_data_structures` to ensure that all the data structures that it depends on are replaced with logically correct versions. This helps isolate the issue with the functions.

In [9]:
function_dependencies_data_structures = {}
function_dependencies_data_structures['star_cell'] = []
function_dependencies_data_structures['get_stars'] = ['Star']
function_dependencies_data_structures['planet_cell'] = []
function_dependencies_data_structures['get_planets'] = ['Planet']
function_dependencies_data_structures['get_all_paths_in'] = []
function_dependencies_data_structures['get_surface_gravity'] = ['all_planets_list', 'planets_list', 'Planet']
function_dependencies_data_structures['get_distances_to_star'] = ['all_planets_list', 'planets_list', 'Planet']
function_dependencies_data_structures['get_liquid_water_distances'] = ['Star', 'stars_dict', 'all_planets_list', 'planets_list', 'Planet']
function_dependencies_data_structures['get_surface_temperatures'] = ['all_planets_list', 'planets_list', 'Planet']

`data_structure_dependencies_functions` stores the previously defined functions that each data structure definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required data structure is used. For these rubric points, when we test a particular data structure, we use `data_structure_dependencies_functions` to ensure that all the functions that it depends on are replaced with logically correct versions. This helps isolate the issue with the data structures.

In [10]:
data_structure_dependencies_functions = {}
data_structure_dependencies_functions['Star'] = []
data_structure_dependencies_functions['stars_dict'] = ['get_stars', 'star_cell']
data_structure_dependencies_functions['Planet'] = []
data_structure_dependencies_functions['planets_list'] = ['get_planets', 'planet_cell']
data_structure_dependencies_functions['star_classes'] = ['get_stars', 'star_cell']
data_structure_dependencies_functions['all_planets_list'] = ['get_planets', 'planet_cell']

`data_structure_dependencies_data_structures` stores the previously defined data structures that each data structure definition accesses. This variable is used for rubric points that the logical correctness of data structures as well as those that check whether a required data structure is used. For these rubric points, when we test a particular data structure, we use `data_structure_dependencies_data_structures` to ensure that all the data structures that it depends on are replaced with logically correct versions. This helps isolate the issue with the data structures.

In [11]:
data_structure_dependencies_data_structures = {}
data_structure_dependencies_data_structures['Star'] = []
data_structure_dependencies_data_structures['stars_dict'] = ['Star']
data_structure_dependencies_data_structures['Planet'] = []
data_structure_dependencies_data_structures['planets_list'] = ['Planet']
data_structure_dependencies_data_structures['star_classes'] = ['Star', 'stars_dict']
data_structure_dependencies_data_structures['all_planets_list'] = ['planets_list', 'Planet'] 

## Functions

Useful functions that are used by many rubric tests can be stored here. The contents of this tag will be executed before each rubric test, so these function definitions get initialized before each rubric test.

`replace_with_false_function` replaces the given `function` with the **false version** of the function, and also replaces all **dependent** functions and data structures with their **true versions**.

In [12]:
def replace_with_false_function(nb, function, false_function):
    nb = replace_defn(nb, function, false_function)
    
    for dependent in function_dependencies_functions.get(function, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in function_dependencies_data_structures.get(function, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
    return nb

`replace_with_false_data_structure` replaces the given `data_structure` with the **false** version of the data structure, and also replaces all **dependent** functions and data structures with their **true versions**.

In [13]:
def replace_with_false_data_structure(nb, data_structure, false_data_structure):
    idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (data_structure))[-1]
    if idx == None:
        idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
    nb = inject_code(nb, idx, false_data_structure)
    nb = remove_initializations(nb, data_structure, start=idx+1)
    
    for dependent in data_structure_dependencies_functions.get(data_structure, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in data_structure_dependencies_data_structures.get(data_structure, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
    return nb

`get_test_text` returns test code that can be readily injected into the notebook. The input should be some code that updates the variable `test_output` and sets its value to be `"All test cases passed!"` when the conditions for passing the rubric test are met. This function will place this code inside a wrapper than ensures that it does not crash the student notebook during execution and also makes the output parsable.

In [14]:
def get_test_text(qnum, test_code):
    test_text = "\"\"\"grader.check('%s')\"\"\"\n\n" % (qnum)
    test_text += "test_output = '%s results: Test crashed!'\n" % (qnum)
    test_text += add_try_except(test_code)
    test_text += "\nprint(test_output)"
    return test_text

`inject_function_logic_check` injects code into the `nb` that detects whether `function` outputs the same as the **true version** of that function (all dependent functions and data structures are also replaced with their **true versions**) on all combinations of inputs from `var_lists`. The comparison between the outputs is performed assuming that the format of the answers is `test_format`.

In [15]:
def inject_function_logic_check(nb, function, var_inputs_code, test_format="TEXT_FORMAT"):
    for dependent in function_dependencies_functions.get(function, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in function_dependencies_data_structures.get(function, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
        
    code = replace_call(true_functions[function], function, "true_"+function)
    code += "\n\n" + verify_fn_defn
    nb = inject_code(nb, len(nb['cells']), code)
    test_code = var_inputs_code + "\n"
    test_code += "test_output = verify_fn(true_%s, %s, var_inputs, '%s')" % (function, function, test_format)
    code = get_test_text(function, test_code)
    nb = inject_code(nb, len(nb['cells']), code)
    return nb

`inject_data_structure_check` injects code into the `nb` that detects whether `data_structure` has the same value as the **true version** of that data structure (all dependent functions and data structures are also replaced with their **true versions**). The comparison between the outputs is performed assuming that the format of the answers is `test_format`.

In [16]:
def inject_data_structure_check(nb, data_structure, test_format="TEXT_FORMAT"):
    for dependent in data_structure_dependencies_functions.get(data_structure, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in data_structure_dependencies_data_structures.get(data_structure, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
        
    code = "import copy\n%s = copy.deepcopy(%s)\n\n" % (data_structure, data_structure)
    code += replace_variable(true_data_structures[data_structure], data_structure, "true_"+data_structure)
    nb = inject_code(nb, len(nb['cells']), code)
    
    test_code = "test_output = '%s results: '" % (data_structure)
    test_code += "+ public_tests.compare(true_%s, %s, '%s')" % (data_structure, data_structure, test_format)
    code = get_test_text(data_structure, test_code)
    nb = inject_code(nb, len(nb['cells']), code)
    return nb

In [17]:
def new_clean_nb(nb):
    return replace_slashes(clean_nb(nb))

## Random Data Generation

Here, functions are defined that can generate **random** data that is in the correct format.

**Warning:** This is the most complex function in the file, and is likely to have some bugs in it. So, **verify** this function **carefully**. The following **requirements** for this function **will not** be met by the function generated by GPT, it is **your responsibility** to modify the function so as to meet these requirements. Otherwise, the datasets are unlikely to produce interesting outputs for the project questions.

Here, functions are defined that can generate **random** data that is in the correct format

* To one of the files `stars_3.csv`, ..., `stars_5.csv`, a star named `GJ 9827` must be added.
* To one of the files `planets_2.csv`, ..., `planets_4.csv`, a planet named `b Cen AB b` must be added.
* To the file `planets_5.csv`, a planet named `Kepler-197 e` must be added.
* To the file `planets_4.csv`, a planet named `HD 20794 d` must be added.

In [18]:
import os
import json
import csv
import random
import string
from shutil import rmtree

def random_data(directory, n=200, break_broken_data=False):
    """
    Generates random datasets inside the given directory.

    :param directory: The path to the project directory.
    :param n: The number of data rows to generate for each file. Default 100.
    """
    # Helper functions
    def normal_distribution_random(min_value, max_value, center, spread, lower_bound, upper_bound):
        while True:
            # Generate a random number with a normal distribution
            num = random.normalvariate(center, spread)
            # Check if number is within the expected 90% range
            if lower_bound <= num <= upper_bound:
                return num
            # If not, regenerate limited to min and max
            if min_value <= num <= max_value:
                return num

    # Create base directories if not there
    data_dir = os.path.join(directory, 'data')
    broken_data_dir = os.path.join(directory, 'broken_data')

    # Spectral types for use in random choice
    spectral_letters = ['A', 'B', 'D', 'F', 'G', 'K', 'L', 'M', 'T', 'W']
    spectral_numbers = [f'{float(num/4):g}' for num in range(0, 40)] + ['']*4
    spectral_romans = ['', '', '', 'I', 'II', 'III', 'IV', 'V', 'VI']
    spectral_types = [f"{letter}{number}{roman}" for letter in spectral_letters for number in spectral_numbers for roman in spectral_romans]
    random.shuffle(spectral_types)
    spectral_types = spectral_types[:(5*n)//2]

    slash_types_1 = [f"{type_1}/{type_2}" for type_1 in spectral_types for type_2 in spectral_types]
    random.shuffle(slash_types_1)
    slash_types_1 = slash_types_1[:len(spectral_types)//8]

    slash_types_2 = [f"{type_1}/{roman}" for type_1 in spectral_types for roman in spectral_romans]
    random.shuffle(slash_types_2)
    slash_types_2 = slash_types_2[:len(spectral_types)//8]

    hyphen_types_1 = [f"{type_1}-{type_2}" for type_1 in spectral_types for type_2 in spectral_types]
    random.shuffle(hyphen_types_1)
    hyphen_types_1 = hyphen_types_1[:len(spectral_types)//8]

    hyphen_types_2 = [f"{type_1}-{roman}" for type_1 in spectral_types for roman in spectral_romans]
    random.shuffle(hyphen_types_2)
    hyphen_types_2 = hyphen_types_2[:len(spectral_types)//8]

    spectral_types.extend(slash_types_1)
    spectral_types.extend(hyphen_types_1)
    spectral_types.extend(slash_types_2)
    spectral_types.extend(hyphen_types_2)
    
    # Discovery methods for use in random choice
    discovery_methods = ['Astrometry', 'Disk Kinematics', 'Eclipse Timing Variations', 'Imaging', 'Microlensing',
                         'Orbital Brightness Modulation', 'Pulsar Timing', 'Pulsation Timing Variations',
                         'Radial Velocity', 'Transit', 'Transit Timing Variations']
    
    # Generate star and planet names
    star_types = ['Kepler', 'TOI', '2MASS', 'HD', 'GJ', 'BD', 'CoRoT', 'EPIC', 'WASP', 'DP'] * 20
    star_types.extend(['alf', 'bet', 'eps', 'gam', 'gam1', 'iot', 'kap', 'mu', 'mu2', 'nu', 'ome', 'omi', 'psi1', 'rho',
                       'tau', 'ups', 'xi'])
    star_numbers = list(range(1, 100))*1000
    star_numbers.extend(list(range(1, 10000))*100)
    star_numbers.extend(list(range(1, 100000)))
    random.shuffle(star_numbers)
    star_numbers = star_numbers[:(5*n)]
    star_names = [f"{star_type}{separator}{number}" for star_type in star_types for separator in ["-", " "] for number in star_numbers]
    random.shuffle(star_names)
    star_names = list(set(star_names))[:5*n]
    
    if 'GJ 9827' in star_names:
        star_names[star_names.index('GJ 9827')] = 'KMT 9827'
    star_names[random.randint(2*n, 4*n-1)] = 'GJ 9827'
    if 'b Cen AB' in star_names:
        star_names[star_names.index('b Cen AB')] = 'b Cen'
    star_names[random.randint(n, 4*n-1)] = 'b Cen AB'
    if 'Kepler-197' in star_names:
        star_names[star_names.index('Kepler-197')] = 'KMT 197'
    star_names[random.randint(4*n, 5*n-1)] = 'Kepler-197'
    if 'HD 20794' in star_names:
        star_names[star_names.index('HD 20794')] = 'KMT 20794'
    star_names[random.randint(3*n, 4*n-1)] = 'HD 20794'

    # CSV file headers
    stars_header = [
        "Star Name", "Spectral Type", "Stellar Effective Temperature [K]",
        "Stellar Radius [Solar Radius]", "Stellar Mass [Solar mass]",
        "Stellar Luminosity [log(Solar)]", "Stellar Surface Gravity [log10(cm/s**2)]",
        "Stellar Age [Gyr]"
    ]
    planets_header = [
        "Planet Name", "Discovery Method", "Discovery Year", "Controversial Flag", 
        "Orbital Period [days]", "Planet Radius [Earth Radius]", "Planet Mass [Earth Mass]",
        "Orbit Semi-Major Axis [au]", "Eccentricity", "Equilibrium Temperature [K]",
        "Insolation Flux [Earth Flux]"
    ]

    # Generate five sets of stars and planets data
    for i in range(1, 6):
        planet_mappings = {}

        stars_path = os.path.join(data_dir, f"stars_{i}.csv")
        planets_path = os.path.join(data_dir, f"planets_{i}.csv")
        mapping_path = os.path.join(data_dir, f"mapping_{i}.json")

        with open(stars_path, mode='w', encoding='utf-8', newline='') as stars_file, \
             open(planets_path, mode='w', encoding='utf-8', newline='') as planets_file:

            stars_writer = csv.DictWriter(stars_file, fieldnames=stars_header)
            planets_writer = csv.DictWriter(planets_file, fieldnames=planets_header)

            stars_writer.writeheader()
            planets_writer.writeheader()

            for j in range(n):
                # Generate star data
                star_name = star_names[(i-1)*n+j]
                star_data = {
                    "Star Name": star_name,
                    "Spectral Type": random.choice(spectral_types),
                    "Stellar Effective Temperature [K]": normal_distribution_random(
                        415.0, 57000.0, 5257.5, 1568.5, 2500.0, 10000.0),
                    "Stellar Radius [Solar Radius]": normal_distribution_random(
                        0.01, 110.0, 25.075, 14.9625, 0.15, 50.0),
                    "Stellar Mass [Solar mass]": normal_distribution_random(
                        0.01, 11.0, 5.505, 3.4975, 0.01, 5.0),
                    "Stellar Luminosity [log(Solar)]": normal_distribution_random(
                        -6.1, 3.8, -0.2, 1.4, -2.2, 2.2),
                    "Stellar Surface Gravity [log10(cm/s**2)]": normal_distribution_random(
                        0.5, 8.0, 3.35, 1.775, 1.5, 5.2),
                    "Stellar Age [Gyr]": normal_distribution_random(
                        0.0, 14.9, 7.001, 4.1995, 0.002, 14.0),
                }
                stars_writer.writerow(star_data)

                # Generate planet data, linking planets to the star
                if random.randint(1, 2) == 1 or star_name in ['b Cen AB', 'Kepler-197', 'HD 20794']:
                    planet_identifiers = list(string.ascii_lowercase)[:i]
                else:
                    planet_identifiers = [str(num) for num in range(1, 27)][:i]
                planet_names = [f"{star_name} {identifier}" for identifier in planet_identifiers]
                for planet_name in planet_names:
                    planet_mappings[planet_name] = star_name
                    planet_data = {
                        "Planet Name": planet_name,
                        "Discovery Method": random.choice(discovery_methods),
                        "Discovery Year": int(normal_distribution_random(
                            1992, 2023, 2019, 2.4, 2015, 2023)),
                        "Controversial Flag": random.choices([0, 1], weights=[75, 25])[0],
                        "Orbital Period [days]": normal_distribution_random(
                            0.09, 402000000, 500.045, 289.9725, 0.09, 1000),
                        "Planet Radius [Earth Radius]": normal_distribution_random(
                            0.3, 77.0, 11.175, 6.4625, 0.35, 22.0),
                        "Planet Mass [Earth Mass]": normal_distribution_random(
                            0.02, 239000, 5001.0, 2843.5, 0.02, 10000),
                        "Orbit Semi-Major Axis [au]": normal_distribution_random(
                            0.0044, 7500.0, 100.0022, 60.00165, 0.01, 200.0),
                        "Eccentricity": normal_distribution_random(
                            0.0, 0.95, 0.05, 0.0285, 0.0, 0.1),
                        "Equilibrium Temperature [K]": normal_distribution_random(
                            35.0, 4500.0, 1100.0, 637.5, 200.0, 2000.0),
                        "Insolation Flux [Earth Flux]": normal_distribution_random(
                            0.0, 44900.0, 6000.0, 635.0, 0.2, 9000.0),
                    }

                    planets_writer.writerow(planet_data)

        with open(mapping_path, 'w', encoding='utf-8') as mapping_file:
            json.dump(planet_mappings, mapping_file)

    # Delete contents of "mapping_5.json"
    mapping_5_path = os.path.join(data_dir, "mapping_5.json")
    with open(mapping_5_path, 'w', encoding='utf-8') as target:
        target.write('}:{')

    # Generate broken_data directory with mappings for planets_5.csv to stars_5.csv
    # Splits mapping from planets_5.csv to different JSON files in broken_data directory
    planets_5_total_mappings = planet_mappings
    broken_data_sub_files = []
    for subdir in os.walk(broken_data_dir):
        for file in subdir[-1]:
            if ".json" in file:
                broken_data_sub_files.append(os.path.join(subdir[0], file))
    if break_broken_data: # Add or delete files to `broken_data` directory when needed
        broken_data_sub_files.pop(random.randint(0, len(broken_data_sub_files)-1))
        broken_data_sub_files.pop(random.randint(0, len(broken_data_sub_files)-1))
        for i in range(5):
            random_dir = os.path.dirname(random.choice(broken_data_sub_files))
            broken_data_sub_files.append(os.path.join(random_dir, 'random.json'))
            
    split_mappings = []
    for i in broken_data_sub_files: # Set up smaller mapping files
        split_mappings.append({}) 
    for i, (planet, star) in enumerate(planets_5_total_mappings.items()):
        split_mappings[i % len(broken_data_sub_files)][planet] = star
    
    if os.path.exists(broken_data_dir):
        rmtree(broken_data_dir)
    os.makedirs(broken_data_dir, exist_ok=True)

    # Generate broken_data subdirectories and mapping json files
    for i, sub_file in enumerate(broken_data_sub_files):
        os.makedirs(os.path.dirname(sub_file), exist_ok=True)

        with open(sub_file, 'w', encoding='utf-8') as file:
            json.dump(split_mappings[i], file)

## True Functions

Here, the **correct** versions of all functions that are defined in the notebook are stored. These functions are compared against the functions in the student notebook to check for their correctness.

In [19]:
true_functions = {}

In [20]:
true_functions['star_cell'] = """
import csv
import os

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data
stars_1_csv = process_csv(os.path.join('data', 'stars_1.csv'))
stars_header = stars_1_csv[0]
stars_1_rows = stars_1_csv[1:]

def star_cell(row_idx, col_name, stars_rows, header=stars_header):
    col_idx = header.index(col_name)
    val = stars_rows[row_idx][col_idx]
    if val == '':
        return None
    elif col_name in ['Stellar Effective Temperature [K]', 'Stellar Radius [Solar Radius]', 'Stellar Mass [Solar mass]', 'Stellar Luminosity [log(Solar)]', 'Stellar Surface Gravity [log10(cm/s**2)]', 'Stellar Age [Gyr]']:
        return float(val)
    else:
        return val"""

In [21]:
true_functions['get_stars'] = """
import csv

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data

def get_stars(star_file):
    stars_data = process_csv(star_file)
    stars_header = stars_data[0]
    stars_rows = stars_data[1:]
    stars = {}
    for row_idx in range(len(stars_rows)):
        star_name = star_cell(row_idx, 'Star Name', stars_rows)
        spectral_type = star_cell(row_idx, 'Spectral Type', stars_rows)
        stellar_effective_temperature = star_cell(row_idx, 'Stellar Effective Temperature [K]', stars_rows)
        stellar_radius = star_cell(row_idx, 'Stellar Radius [Solar Radius]', stars_rows)
        stellar_mass = star_cell(row_idx, 'Stellar Mass [Solar mass]', stars_rows)
        stellar_luminosity = star_cell(row_idx, 'Stellar Luminosity [log(Solar)]', stars_rows)
        stellar_surface_gravity = star_cell(row_idx, 'Stellar Surface Gravity [log10(cm/s**2)]', stars_rows)
        stellar_age = star_cell(row_idx, 'Stellar Age [Gyr]', stars_rows)
        star = Star(spectral_type, stellar_effective_temperature, stellar_radius, stellar_mass, stellar_luminosity, stellar_surface_gravity, stellar_age)
        stars[star_name] = star
    return stars"""

In [22]:
true_functions['planet_cell'] = """
import csv
import os

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data
planets_1_csv = process_csv(os.path.join('data', 'planets_1.csv'))
planets_header = planets_1_csv[0]
planets_1_rows = planets_1_csv[1:]

def planet_cell(row_idx, col_name, planets_rows, header=planets_header):
    col_idx = header.index(col_name)
    val = planets_rows[row_idx][col_idx]
    if val == '':
        return None
    if col_name in ['Controversial Flag']:
        if val == '1':
            return True
        else:
            return False
    elif col_name in ['Discovery Year']:
        return int(val)
    elif col_name in ['Orbital Period [days]', 'Planet Radius [Earth Radius]', 'Planet Mass [Earth Mass]', 'Orbit Semi-Major Axis [au]', 'Eccentricity', 'Equilibrium Temperature [K]', 'Insolation Flux [Earth Flux]']:
        return float(val)
    else:
        return val"""

In [23]:
true_functions['get_planets'] = """
import csv
import json

def read_json(path):
    with open(path, encoding='utf-8') as f:
        return json.load(f)

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data

def get_planets(planet_file, mapping_file):
    planets = []
    try:
        mapping_dict = read_json(mapping_file)
    except json.JSONDecodeError:
        return []
    planets_csv = process_csv(planet_file)
    planets_header = planets_csv[0]
    planets_rows = planets_csv[1:]
    for row_idx in range(len(planets_rows)):
        try:
            planet_name = planet_cell(row_idx, 'Planet Name', planets_rows)
            host_name = mapping_dict[planet_name]
            discovery_method = planet_cell(row_idx, 'Discovery Method', planets_rows)
            discovery_year = planet_cell(row_idx, 'Discovery Year', planets_rows)
            controversial_flag = planet_cell(row_idx, 'Controversial Flag', planets_rows)
            orbital_period = planet_cell(row_idx, 'Orbital Period [days]', planets_rows)
            planet_radius = planet_cell(row_idx, 'Planet Radius [Earth Radius]', planets_rows)
            planet_mass = planet_cell(row_idx, 'Planet Mass [Earth Mass]', planets_rows)
            semi_major_radius = planet_cell(row_idx, 'Orbit Semi-Major Axis [au]', planets_rows)
            eccentricity = planet_cell(row_idx, 'Eccentricity', planets_rows)
            equilibrium_temperature = planet_cell(row_idx, 'Equilibrium Temperature [K]', planets_rows)
            insolation_flux = planet_cell(row_idx, 'Insolation Flux [Earth Flux]', planets_rows)
            planet = Planet(planet_name, host_name, discovery_method, discovery_year, controversial_flag, orbital_period, planet_radius, planet_mass, semi_major_radius, eccentricity, equilibrium_temperature, insolation_flux)
            planets.append(planet)
        except IndexError:
            continue
        except ValueError:
            continue
        except KeyError:
            continue
    return planets"""

In [24]:
true_functions['plot_scatter'] = """
import matplotlib.pyplot as plt

def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    plt.scatter(x_data, y_data, c=c, s=s)
    plt.xlabel(x_label)
    plt.ylabel(y_label)"""

In [25]:
true_functions['plot_scatter_multiple'] = """
import matplotlib.pyplot as plt

def plot_scatter_multiple(x_data_dict, y_data_dict, x_label='x axis', y_label='y axis'):
    legend_values = list(x_data_dict.keys())
    for key in x_data_dict:
        plt.scatter(x_data_dict[key], y_data_dict[key], s=7)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(legend_values)"""

In [26]:
true_functions['get_all_paths_in'] = """
import os

def get_all_paths_in(directory):
    paths = []
    files = os.listdir(directory)
    for file in files:
        if file.startswith('.'):
            continue
        path = os.path.join(directory, file)
        if os.path.isfile(path):
            paths.append(path)
        elif os.path.isdir(path):
            paths.extend(get_all_paths_in(path))
    return sorted(paths, key=lambda path: path.split(os.path.sep), reverse=True)"""

In [27]:
true_functions['get_surface_gravity'] = """
def get_surface_gravity(planet):
    mass = planet.planet_mass
    radius = planet.planet_radius
    if mass == None or radius == None:
        return None
    return mass / radius ** 2"""

In [28]:
true_functions['get_distances_to_star'] = """
def get_distances_to_star(planet):
    ecc = planet.eccentricity
    semi_major = planet.semi_major_radius
    if ecc == None or semi_major == None:
        return None
    return [semi_major * (1 - abs(ecc)), semi_major * (1 + abs(ecc))]"""

In [29]:
true_functions['get_liquid_water_distances'] = """
import csv

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data

def get_liquid_water_distances(planet):
    lum = stars_dict[planet.host_name].stellar_luminosity
    if lum == None:
        return None
    return [(10 ** lum / 1.15) ** 0.5, (10 ** lum / 0.53) ** 0.5]"""

In [30]:
true_functions['get_surface_temperatures'] = """
def get_surface_temperatures(planet):
    temp = planet.equilibrium_temperature
    if temp == None:
        return None
    return [0.5 ** (1 / 4) * temp, temp]"""

## True Data Structures

Here, the **correct** versions of all data structures that are defined in the notebook are stored. These data structures are compared against the data structures in the student notebook to check for their correctness.

In [31]:
true_data_structures = {}

In [32]:
true_data_structures['Star'] = """
from collections import namedtuple
star_attributes = ['spectral_type', 'stellar_effective_temperature', 'stellar_radius', 'stellar_mass', 'stellar_luminosity', 'stellar_surface_gravity', 'stellar_age']
Star = namedtuple('Star', star_attributes)"""

In [33]:
true_data_structures['stars_dict'] = """
import csv
import os

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data
stars_dict = {}
for i in range(1, 6):
    csv_file = os.path.join('data', 'stars_%d.csv' % i)
    curr_stars_dict = get_stars(csv_file)
    stars_dict.update(curr_stars_dict)
len(stars_dict)"""

In [34]:
true_data_structures['Planet'] = """
from collections import namedtuple
planets_attributes = ['planet_name', 'host_name', 'discovery_method', 'discovery_year', 'controversial_flag', 'orbital_period', 'planet_radius', 'planet_mass', 'semi_major_radius', 'eccentricity', 'equilibrium_temperature', 'insolation_flux']
Planet = namedtuple('Planet', planets_attributes)"""

In [35]:
true_data_structures['planets_list'] = """
import csv
import os
import json

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data

def read_json(path):
    with open(path, encoding='utf-8') as f:
        return json.load(f)
planets_list = []
for i in range(1, 6):
    planet_path = os.path.join('data', 'planets_%d.csv' % i)
    mapping_path = os.path.join('data', 'mapping_%d.json' % i)
    new_planets = get_planets(planet_path, mapping_path)
    if new_planets in [[], None, {}]:
        continue
    planets_list.extend(get_planets(planet_path, mapping_path))
len(planets_list)"""

In [36]:
true_data_structures['star_classes'] = """
import csv

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data
star_classes = {'Red Giant': [], 'White Dwarf': [], 'Neutron Star': []}
for star in stars_dict:
    star_info = stars_dict[star]
    if star_info.stellar_mass == None:
        continue
    if 0.3 <= star_info.stellar_mass < 8:
        star_classes['Red Giant'].append(star)
    elif 8 <= star_info.stellar_mass < 10.5:
        star_classes['White Dwarf'].append(star)
    elif star_info.stellar_mass >= 10.5:
        star_classes['Neutron Star'].append(star)"""

In [37]:
true_data_structures['all_planets_list'] = """
import csv
import os
import json

def process_csv(filename):
    with open(filename) as file:
        csv_reader = csv.reader(file)
        list_data = list(csv_reader)
        return list_data

def read_json(path):
    with open(path, encoding='utf-8') as f:
        return json.load(f)
all_planets_list = []
all_planets_list.extend(planets_list)
for json_path in broken_data:
    all_planets_list.extend(get_planets(os.path.join('data', 'planets_5.csv'), json_path))
len(all_planets_list)"""

## Original

The original test simply runs the student's notebook as it is (after removing cells with syntax errors, and performing other clean-up). This helps us detect if the student failed any public tests.

In [38]:
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))

results['original'] = parse_nb(run_nb(nb, os.path.join(DIRECTORY, "hidden", "original", FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


## Hardcode

The hardcode tests run the student's notebook on different datasets. However, `public_tests.py` remains unchanged. So, if the answers are hardcoded in the student's notebook, we expect their code to still pass the public tests on all the different datasets. If their code fails any one of the different hardcode datasets, we take that to mean that the answer is not hardcoded.

In [39]:
for subdirectory in os.listdir(os.path.join(DIRECTORY, "hidden", "hardcode")):
    path = os.path.join(DIRECTORY, "hidden", "hardcode", subdirectory)
    good_dataset = False
    while not good_dataset:
        if os.path.exists(os.path.join(path, FILE)):
            nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
        hardcode_results = parse_nb(run_nb(nb, os.path.join(path, FILE)))
        good_dataset = True
        for qnum in hardcode_results:
            if qnum.startswith('q') and hardcode_results[qnum] == 'All test cases passed!':
                print(qnum + ' failed!')
                good_dataset = False
                break
        if not good_dataset:
            random_data(path, 200, True)
    print(subdirectory + ' done!')

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


1 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


3 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


2 done!


In [40]:
for hardcode in os.listdir(os.path.join(DIRECTORY, "hidden", "hardcode")):
    nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
    results['hardcode: ' + hardcode] = parse_nb(run_nb(nb, os.path.join(DIRECTORY, "hidden", "hardcode", hardcode, FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


## Rubric Tests

The tests for the rubric points will be defined below. Only the code inside the tags will be executed by `hidden_tests.py`, so the code outside the tags are used for generating the hidden datasets in the first place.

### Instructions for creating rubric tests:

Functions inside `hidden_tests.py` can be used to modify the student notebook, before executing and parsing the outputs. It is recommended that before trying to create rubric tests, a user goes through all the functions inside `hidden_tests.py` first. Here is a list of commonly used functions that will be most useful:

* **`read_nb`**: `read_nb(file)` **reads** a `file` in the `.ipynb` file format and returns a `nb`.
* **`run_nb`**: `run_nb(nb, file)` **executes** `nb` at the location `file` and **writes** the contents back into `file`.
* **`parse_nb`**: `parse_nb(nb)` read the contents of a student `nb` and **extracts** all graded questions and answers.
* **`truncate_nb`**: `truncate_nb(nb, start, end)` takes in a `nb`, and returns a **sliced** notebook between the cells indexed `start` and `end`.
* **`find_all_cell_indices`**: `find_all_cell_indices(nb, cell_type, marker)` returns **all** the indices in `nb` of cell type `cell_type` that **contains** the `marker` in its source.
* **`inject_code`**: `inject_code(nb, idx, code)` creates a **new** code cell in `nb` **after** the index `idx` with `code` in it.
* **`count_defns`**: `count_defns(nb, func_name)` **counts** the number of times `func_name` is defined in the `nb`.
* **`replace_defn`**: `replace_defn(nb, func_name, new_defn)` **replaces** the definition of `func_name` in `nb` with `new_defn`.
* **`replace_call`**: `replace_call(text, func_name, new_name)` **replaces** all **calls** and definition **names** to `func_name` with `new_name` in `text`.
* **`find_code`**: `find_code(nb, target)` returns the **number** of times that the **text** `target` appears in a code cell in `nb`.
* **`replace_code`**: `replace_code(nb, target, new_code, start, end)` **replaces** all instances of the **text** `target` in a code cell between the indices `start` and `end` with the **text** `new_code`.
* **`add_try_except`**: `add_try_except(text)` adds a (bare) **try/except block** around any given block of code.
* **`detect_restart_and_run_all`**: `detect_restart_and_run_all(nb)` flags if any **non-empty code cell** in `nb` is **not executed**.
* **`detect_imports`**: `detect_imports(nb)` returns a list of **all** the **import** statements in the `nb`.
* **`detect_ast_objects`**: `detect_ast_objects(nb, objects)` returns a dict of **all** cells in the `nb` with the **ast objects** `objects` in them.
* **`get_first_plot`**: `get_first_plot(nb, image_file)` returns the first **image** found in the output of a code cell in `nb`, and also stores it in `image_file` for reference.
* **`get_label_plot`**: `get_label_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing just the **label** at the location indicated by `kind` - `"left"`, `"right"`, `"top"`, or `"bottom"`.
* **`get_without_label_plot`**: `get_without_label_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing everything **except** the **label** at the location indicated by `kind` - `"left"`, `"right"`, `"top"`, or `"bottom"`.
* **`get_ticks_plot`**: `get_ticks_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing just the **ticks** at the location indicated by `kind` - `"left"`, or `"bottom"`.
* **`get_without_ticks_plot`**: `get_without_ticks_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing everything **except** the **ticks** at the location indicated by `kind` - `"left"`, or `"bottom"`.
* **`get_bounding_box_plot`**: `get_bounding_box_plot(plot)` **crops** the `plot` and returns returns a plot containing just the **bounding box** of the plot.
* **`check_text_in_plot`**: `check_text_in_plot(plot, expected_text)` checks if the `expected_text` is in the `plot`, and returns both the **missing** and the **extra** text in the given `plot`.

### q1: incorrect logic is used to answer

In [41]:
rubric_item = 'q1: incorrect logic is used to answer'
readme_text = """Check that your code correctly identifies all
planets orbiting 'GJ 9827'. Ensure that you are
matching planets to the star based on the correct
data fields. If you are modifying the data,
confirm your modifications are accurate. Utilize
the provided solution's structure and verify the
implementation of the mathematical operations for
the calculation of the ratio. Test your code with
different datasets to ensure its robustness."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [42]:
random_data(directories[rubric_item], 200)

In [43]:
rubric_item = 'q1: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q1')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [44]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q1: `planets_list` data structure is not used to answer

In [45]:
rubric_item = 'q1: `planets_list` data structure is not used to answer'
readme_text = """Review your code to ensure that you are using the
`planets_list` data structure provided for the
computation of the ratios. Check that you are
correctly accessing the attributes of each planet
within this list and performing the calculations
as stipulated in the question. Also, ensure that
your results are stored in a list of floats as
required. Consider running your code on various
inputs to cover different scenarios and catch
potential errors."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [46]:
random_data(directories[rubric_item], 10)

In [47]:
rubric_item = 'q1: `planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q1')")[-1])

new_planets_list = true_data_structures["planets_list"] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
planets_list = [Planet(*planet) for planet in raw_planets]'''
nb = replace_with_false_data_structure(nb, 'planets_list', new_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [48]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q2: incorrect logic is used to answer

In [49]:
rubric_item = 'q2: incorrect logic is used to answer'
readme_text = """Ensure that your code correctly computes the
coefficient of variance using the standard
deviation and mean of the list `ratios_gj9827`.
Review your implementation for any possible
logical errors. To troubleshoot, verify if you are
correctly handling import statements and using the
`stdev` and `mean` functions from the `statistics`
module as intended. Double-check your calculations
to ensure accurate results with modified datasets."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [50]:
random_data(directories[rubric_item], 200)

In [51]:
rubric_item = 'q2: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q2')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [52]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q2: recomputed variable defined in Question 1

In [53]:
rubric_item = 'q2: recomputed variable defined in Question 1'
readme_text = """The test checks if you reused the variable
`ratios_gj9827` correctly, without redefining or
recomputing it after Question 1. Verify that your
solution uses this variable directly when
calculating the coefficient of variance. Ensure
that your import statement for the `statistics`
module is correct and that you are calling the
`stdev` and `mean` functions properly on the
`ratios_gj9827` list. If the list contains
negative numbers, the standard deviation and mean
calculations will still be valid, but the nature
of the data might affect the coefficient of
variance. Make sure to handle any list data type
effectively."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [54]:
random_data(directories[rubric_item], 10)

In [55]:
rubric_item = 'q2: recomputed variable defined in Question 1'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q2')")[-1])

inject_code_after_q1 = '''
ratios_gj9827 = [-1, -2, -3, -5, -10]
'''
q1_end_idx = find_all_cell_indices(nb, "code", "grader.check('q1')")[-1]
nb = inject_code(nb, q1_end_idx + 1, inject_code_after_q1)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [56]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q3: incorrect logic is used to answer

In [57]:
rubric_item = 'q3: incorrect logic is used to answer'
readme_text = """Check the code's logic for calculating the
predicted and actual stellar mass, and ensure it
uses the `mean` of ratios from the correct
question and the `Star` object's attribute for
actual stellar mass. Ensure that the percentage
change formula is correctly implemented as
described."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [58]:
random_data(directories[rubric_item], 200)

In [59]:
rubric_item = 'q3: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q3')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [60]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q3: recomputed variable defined in Question 1

In [61]:
rubric_item = 'q3: recomputed variable defined in Question 1'
readme_text = """Check usage of previously computed variable
`ratios_gj9827`. Ensure no redefinition occurs
that could alter its value before performing
calculations. Use appropriate mean computation
method and consider negative values' impact on the
mean if they exist."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [62]:
random_data(directories[rubric_item], 200)

In [63]:
rubric_item = 'q3: recomputed variable defined in Question 1'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

start_q3 = find_all_cell_indices(nb, "markdown", '**Question 3:**')[-1]
end_q3 = find_all_cell_indices(nb, "code", "grader.check('q3')")[-1]
nb = truncate_nb(nb, end=end_q3)

inject_code_after_q1 = find_all_cell_indices(nb, "code", "grader.check('q1')")[-1] + 1
code_to_inject = '''
ratios_gj9827 = [-1, -2, -3]  # This makes no physical sense, but is needed for the test
'''
nb = inject_code(nb, inject_code_after_q1, code_to_inject)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [64]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q3: `stars_dict` data structure is not used to answer

In [65]:
rubric_item = 'q3: `stars_dict` data structure is not used to answer'
readme_text = """Ensure the `stars_dict` data structure is used to
access the actual `stellar_mass` of the star 'GJ
9827'. Data structures provided should be utilized
efficiently as they are preprocessed to contain
necessary information. Check for correct attribute
access and proper calculation using the given
formula."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [66]:
rubric_item = 'q3: `stars_dict` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q3')")[-1])

false_stars_dict = true_data_structures["stars_dict"] + '''
import random

random.seed(0)
stars_dict_keys = list(stars_dict.keys())
stars_dict_values = list(stars_dict.values())
random.shuffle(stars_dict_keys)
random.shuffle(stars_dict_values)

stars_dict = {}
for i in range(len(stars_dict_keys)):
    stars_dict[stars_dict_keys[i]] = stars_dict_values[i]
'''
nb = replace_with_false_data_structure(nb, 'stars_dict', false_stars_dict)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [67]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q4: incorrect logic is used to answer

In [68]:
rubric_item = 'q4: incorrect logic is used to answer'
readme_text = """Ensure your logic properly checks for missing
values and filters out planets without complete
data. Remember to skip planets with `None` type or
zero values before appending to your lists. Review
conditional statements to correct the issue."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [69]:
import os
import csv
import json
import random


def modify_data(directory):
    """
    Modify the datasets in the given directory to comply with the test requirements.
    
    Args:
    - directory: The path to the project directory containing the data.
    """
    # Get the paths to all planet CSV files
    planet_files = [f for f in os.listdir(os.path.join(directory, 'data')) if f.startswith('planets_')]
    
    # Iterate through each planet file
    for planet_file in planet_files:
        planet_path = os.path.join(directory, 'data', planet_file)
        
        # Read the data from the CSV file and modify values based on the conditions
        modified_rows = []
        with open(planet_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # With probability of 1/3, delete or set to 0 the 'Insolation Flux [Earth Flux]' value
                if random.choice([0, 1, 2]) == 0:
                    row['Insolation Flux [Earth Flux]'] = ''
                elif random.choice([0, 1, 2]) == 0:
                    row['Insolation Flux [Earth Flux]'] = '0'

                # With probability of 1/3, delete or set to 0 the 'Equilibrium Temperature [K]' value
                if random.choice([0, 1, 2]) == 0:
                    row['Equilibrium Temperature [K]'] = ''
                elif random.choice([0, 1, 2]) == 0:
                    row['Equilibrium Temperature [K]'] = '0'
                
                modified_rows.append(row)
        
        # Write the modified data back to the CSV file
        with open(planet_path, 'w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(modified_rows)

# First, we completely modify the dataset as per the rubric being tested
# The 'directories' variable and 'rubric_item' must have been defined before calling this
random_data(directories[rubric_item])

# Now call the function that will modify the data to fit the specific test conditions
modify_data(directories[rubric_item])

In [70]:
rubric_item = 'q4: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q4')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [71]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q4: `planets_list` data structure is not used to answer

In [72]:
rubric_item = 'q4: `planets_list` data structure is not used to answer'
readme_text = """Ensure you are using the `planets_list` data
structure provided to extract necessary data.
Review its usage and verify you are not
re-processing data or using an alternative data
structure."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [73]:
import os
import csv
import json
import random


def modify_data(directory):
    """
    Modify the datasets in the given directory to comply with the test requirements.
    
    Args:
    - directory: The path to the project directory containing the data.
    """
    # Get the paths to all planet CSV files
    planet_files = [f for f in os.listdir(os.path.join(directory, 'data')) if f.startswith('planets_')]
    
    # Iterate through each planet file
    for planet_file in planet_files:
        planet_path = os.path.join(directory, 'data', planet_file)
        
        # Read the data from the CSV file and modify values based on the conditions
        modified_rows = []
        with open(planet_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # With probability of 1/3, delete or set to 0 the 'Insolation Flux [Earth Flux]' value
                if random.choice([0, 1, 2]) == 0:
                    row['Insolation Flux [Earth Flux]'] = ''
                elif random.choice([0, 1, 2]) == 0:
                    row['Insolation Flux [Earth Flux]'] = '0'

                # With probability of 1/3, delete or set to 0 the 'Equilibrium Temperature [K]' value
                if random.choice([0, 1, 2]) == 0:
                    row['Equilibrium Temperature [K]'] = ''
                elif random.choice([0, 1, 2]) == 0:
                    row['Equilibrium Temperature [K]'] = '0'
                
                modified_rows.append(row)
        
        # Write the modified data back to the CSV file
        with open(planet_path, 'w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(modified_rows)

# First, we completely modify the dataset as per the rubric being tested
# The 'directories' variable and 'rubric_item' must have been defined before calling this
random_data(directories[rubric_item])

# Now call the function that will modify the data to fit the specific test conditions
modify_data(directories[rubric_item])

In [74]:
rubric_item = 'q4: `planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_index = find_all_cell_indices(nb, "code", "grader.check('q4')")[-1]
nb = truncate_nb(nb, end=end_index)

new_planets_list = true_data_structures["planets_list"] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
planets_list = [Planet(*planet) for planet in raw_planets]'''
nb = replace_with_false_data_structure(nb, 'planets_list', new_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [75]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q4: plot is incorrect

In [76]:
rubric_item = 'q4: plot is incorrect'
readme_text = """Ensure `plot_scatter` is called with the lists you
constructed, with `insolation_flux` on the x-axis
and the fourth power of `equilibrium_temperature`
on the y-axis. Check if `plot_scatter` is
correctly implemented and called where the plot is
expected."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [77]:
rubric_item = 'q4: plot is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q4')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 4:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

# Injecting new global variable before the question starts
plot_dict_code = '''
hidn_plot_dict = {}

import matplotlib.pyplot as plt    
    
def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    global hidn_plot_dict
    hidn_plot_dict['x'] = x_data
    hidn_plot_dict['y'] = y_data
'''
start_index = find_all_cell_indices(nb, "markdown", '**Question 4:**')[-1]+1
nb = inject_code(nb, start_index, plot_dict_code)

# Injecting test code at the end of the notebook to compare contents of hidn_plot_dict to the expected values
test_code = '''
expected_plots = public_tests.get_expected_plots()
expected_flux = expected_plots['q4']['flux_list']
expected_temp_4th_power = expected_plots['q4']['temp_4th_power_list']

if (public_tests.compare(expected_flux, hidn_plot_dict.get('x', []), "TEXT_FORMAT_ORDERED_LIST") == public_tests.PASS and
    public_tests.compare(expected_temp_4th_power, hidn_plot_dict.get('y', []), "TEXT_FORMAT_ORDERED_LIST") == public_tests.PASS):
    test_output = 'q4 results: ' + public_tests.PASS
'''
nb = inject_code(nb, len(nb['cells']), get_test_text("q4", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
    comments[rubric_item] += '\nFAILED TEST: ' + results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q4: plot is not properly labeled

In [78]:
rubric_item = "q4: plot is not properly labeled"
readme_text = """The notebook is parsed and the
plot for Question 4 is identified.
The image is parsed, and the text
in the left and bottom of the image
are read.
If any of the required labels are
missing, then the test is deemed
a failure.""" 

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [79]:
rubric_item = 'q4: plot is not properly labeled'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q4')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 4:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

hidn_plot_title_dict_code = '''
hidn_plot_title_dict = {}

import matplotlib.pyplot as plt    
    
def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    global hidn_plot_title_dict
    hidn_plot_title_dict['x'] = x_label
    hidn_plot_title_dict['y'] = y_label
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", "**Question 4:**")[-1]+1, hidn_plot_title_dict_code)

test_code = '''
expected_x_label = 'Insolation Flux'
expected_y_label = '(Equilibrium Temperature)**4'
actual_x_label = hidn_plot_title_dict.get('x', None)
actual_y_label = hidn_plot_title_dict.get('y', None)

test_output = 'q4 results:'

if public_tests.compare(expected_x_label, actual_x_label, "TEXT_FORMAT") == public_tests.PASS:
    if public_tests.compare(expected_y_label, actual_y_label, "TEXT_FORMAT") == public_tests.PASS:
        test_output = 'q4 results: ' + public_tests.PASS
    else:
        test_output += ' expected '  + repr(expected_y_label) + ' but found ' + repr(actual_y_label)
else:
    test_output += ' expected '  + repr(expected_x_label) + ' but found ' + repr(actual_x_label)
'''
nb = inject_code(nb, len(nb['cells']), get_test_text("q4", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'

test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q5: incorrect comparison operator is used

In [80]:
rubric_item = 'q5: incorrect comparison operator is used'
readme_text = """Check your comparison for `insolation_flux`
values. Ensure you are using the correct operator
to exclude values over 7000. Also, confirm you're
correctly handling missing data. Review the use of
comparison operators and conditional statements."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [81]:
import os
import json
import csv
import random

def modify_data(directory):
    # Iterate over the 'planets_*.csv' files in the 'data' directory
    for i in range(1, 6):
        planet_file_name = f'planets_{i}.csv'
        planet_file_path = os.path.join(directory, 'data', planet_file_name)
        
        # Read the existing contents of the file
        with open(planet_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            planets_data = list(reader)
        
        # Modify some rows to set 'Insolation Flux [Earth Flux]' to 7000 with 1/3 probability
        for row in planets_data:
            if random.random() < 1/3:  # 1/3 probability
                row['Insolation Flux [Earth Flux]'] = '7000.0'
        
        # Write the modified contents back to the file
        with open(planet_file_path, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames, lineterminator='\n')
            writer.writeheader()
            writer.writerows(planets_data)

# The 'directories' and 'rubric_item' variables are predefined,
# as is the 'random_data' function that generates random data.

# Completely modify the dataset.
random_data(directories[rubric_item])  # Call without specifying size to use default size

# Call the modify_data function with the project directory path.
modify_data(directories[rubric_item])

In [82]:
rubric_item = 'q5: incorrect comparison operator is used'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q5')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [83]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q5: `planets_list` data structure is not used to answer

In [84]:
rubric_item = 'q5: `planets_list` data structure is not used to answer'
readme_text = """Check that you're using the `planets_list` data
structure provided in the course materials. Ensure
your code filters out planets with missing data
and properly excludes planets with
`insolation_flux` over 7000. Use list
comprehensions or loops to create the required
lists for plotting."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [85]:
rubric_item = 'q5: `planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q5')")[-1])

false_planets_list = true_data_structures["planets_list"] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
planets_list = [Planet(*planet) for planet in raw_planets]'''
nb = replace_with_false_data_structure(nb, 'planets_list', false_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [86]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q5: plot is incorrect

In [87]:
rubric_item = 'q5: plot is incorrect'
readme_text = """Ensure you're plotting the correct data, exclude
insolation_flux > 7000, and check for missing
values. Verify the usage of the plot_scatter
function and confirm the plot's visibility in the
assigned cell."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [88]:
rubric_item = 'q5: plot is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q5')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 5:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

hidn_plot_dict_code = '''
hidn_plot_dict = {}

import matplotlib.pyplot as plt    
    
def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    global hidn_plot_dict
    hidn_plot_dict['x'] = x_data
    hidn_plot_dict['y'] = y_data
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", "**Question 5:**")[-1]+1, hidn_plot_dict_code)

test_code = '''
expected_plots = public_tests.get_expected_plots()
expected_flux_list = expected_plots['q5']['flux_list_no_outliers']
expected_temp_4th_power_list = expected_plots['q5']['temp_4th_power_list_no_outliers']

if public_tests.compare(expected_flux_list, hidn_plot_dict.get('x'), "TEXT_FORMAT_ORDERED_LIST") == public_tests.PASS:
    if public_tests.compare(expected_temp_4th_power_list, hidn_plot_dict.get('y'), "TEXT_FORMAT_ORDERED_LIST") == public_tests.PASS:
        test_output = 'q5 results: ' + public_tests.PASS
'''
nb = inject_code(nb, len(nb['cells']), get_test_text("q5", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
    comments[rubric_item] += '\nFAILED TEST: ' + results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [89]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q5: plot is not properly labeled

In [90]:
rubric_item = "q5: plot is not properly labeled"
readme_text = """The notebook is parsed and the
plot for Question 5 is identified.
The image is parsed, and the text
in the left and bottom of the image
are read.
If any of the required labels are
missing, then the test is deemed
a failure.""" 

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [91]:
rubric_item = 'q5: plot is not properly labeled'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q5')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 5:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

hidn_plot_title_dict_code = '''
hidn_plot_title_dict = {}

import matplotlib.pyplot as plt    
    
def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    global hidn_plot_title_dict
    hidn_plot_title_dict['x'] = x_label
    hidn_plot_title_dict['y'] = y_label
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", "**Question 5:**")[-1]+1, hidn_plot_title_dict_code)

test_code = '''
expected_x_label = 'Insolation Flux'
expected_y_label = '(Equilibrium Temperature)**4'
actual_x_label = hidn_plot_title_dict.get('x', None)
actual_y_label = hidn_plot_title_dict.get('y', None)

test_output = 'q5 results:'

if public_tests.compare(expected_x_label, actual_x_label, "TEXT_FORMAT") == public_tests.PASS:
    if public_tests.compare(expected_y_label, actual_y_label, "TEXT_FORMAT") == public_tests.PASS:
        test_output = 'q5 results: ' + public_tests.PASS
    else:
        test_output += ' expected '  + repr(expected_y_label) + ' but found ' + repr(actual_y_label)
else:
    test_output += ' expected '  + repr(expected_x_label) + ' but found ' + repr(actual_x_label)
'''
nb = inject_code(nb, len(nb['cells']), get_test_text("q5", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'

test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### star_classes: data structure is defined incorrectly

In [92]:
rubric_item = 'star_classes: data structure is defined incorrectly'
readme_text = """Ensure that your `star_classes` dictionary uses
the correct keys and that you are appending star
names to the appropriate lists based on the
`stellar_mass`. Review the hints provided in the
question to classify the stars accurately, and
make sure you've correctly initialized the
dictionary as required."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [93]:
rubric_item = 'star_classes: data structure is defined incorrectly'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

# Find the cell index after the definition of 'star_classes'
end_idx = find_all_cell_indices(nb, "code", "grader.check('star_classes')")[-1]

# Truncate the notebook to that cell index to save compute
nb = truncate_nb(nb, end=end_idx)

# Inject data_structure check code for 'star_classes'
nb = inject_data_structure_check(nb, 'star_classes', "TEXT_FORMAT_DICT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [94]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### star_classes: incorrect comparison operators are used

In [95]:
rubric_item = 'star_classes: incorrect comparison operators are used'
readme_text = """Ensure you have used the correct comparison
operators to classify star masses (greater than,
less than, etc.). Double-check the boundaries
between classes (e.g., 0.3, 8, and 10.5 solar
masses) and include the correct stars in the
corresponding lists."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [96]:
import os
import pandas as pd
import json

def modify_data(directory):
    # Paths for the stars files in the data directory
    stars_files = [f'stars_{i}.csv' for i in range(1, 6)]

    # Modify each stars file
    for filename in stars_files:
        file_path = os.path.join(directory, 'data', filename)
        # Read the data from the csv file
        stars_data = pd.read_csv(file_path, encoding='utf-8')
        
        # Get the number of rows that need to be modified to half the data
        num_rows = len(stars_data) // 2
        
        # Change 'Stellar Mass [Solar mass]' to either 0.3, 8, or 10.5 for half the rows
        choices = [0.3, 8, 10.5]
        mass_modifications = [choices[i % 3] for i in range(num_rows)]
        
        # Apply the modifications randomly to half the rows
        rows_to_modify = stars_data.sample(num_rows).index
        stars_data.loc[rows_to_modify, 'Stellar Mass [Solar mass]'] = mass_modifications
        
        # Save the modified data back to the file
        stars_data.to_csv(file_path, index=False, encoding='utf-8')

# Assume 'directories' and 'rubric_item' have been defined
# Generate completely random datasets as per test details and call the modify_data function
random_data(directories[rubric_item])  # Call random_data function with default size
modify_data(directories[rubric_item])  # Modify the data using the directory of the current rubric item

In [97]:
rubric_item = 'star_classes: incorrect comparison operators are used'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('star_classes')")[-1])

# Inject data_structure check code for 'star_classes'
nb = inject_data_structure_check(nb, 'star_classes', "TEXT_FORMAT_DICT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [98]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### star_classes: `stars_dict` data structure is not used

In [99]:
rubric_item = 'star_classes: `stars_dict` data structure is not used'
readme_text = """Check that your code is utilizing the provided
`stars_dict` as expected. Ensure that you
reference its keys and associated `Star` object
attributes correctly when classifying stars, and
that your conditions for classification align with
the prompts given. Make sure you are not
redefining `stars_dict` or using an outdated
version of it."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [100]:
random_data(directories[rubric_item], 10)

In [101]:
rubric_item = 'star_classes: `stars_dict` data structure is not used'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_idx = find_all_cell_indices(nb, "code", "grader.check('star_classes')")[-1]
nb = truncate_nb(nb, end=end_idx)

# Inject data_structure check code for 'star_classes'
nb = inject_data_structure_check(nb, 'star_classes', "TEXT_FORMAT_DICT")

new_stars_dict = true_data_structures["stars_dict"] + '''
import random

random.seed(0)
stars_dict_keys = list(stars_dict.keys())
stars_dict_values = list(stars_dict.values())
random.shuffle(stars_dict_keys)
random.shuffle(stars_dict_values)

stars_dict = {}
for i in range(len(stars_dict_keys)):
    stars_dict[stars_dict_keys[i]] = stars_dict_values[i]
'''
nb = replace_with_false_data_structure(nb, 'stars_dict', new_stars_dict)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [102]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q6: incorrect logic is used to answer

In [103]:
rubric_item = 'q6: incorrect logic is used to answer'
readme_text = """Verify handling of missing data and calculations
within your logic. Ensure that zeros are not
treated as missing and averaging only considers
present values. Review dataset alterations and
retest accordingly."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [104]:
import os
import csv
import json
import random

def modify_data(directory):
    # Define the filenames for stars CSV files
    stars_files = [f'stars_{i}.csv' for i in range(1, 6)]

    for filename in stars_files:
        # Construct the full path to the current stars file
        file_path = os.path.join(directory, 'data', filename)
        # List to store modified rows
        modified_data = []

        # Open the current stars CSV file
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Randomly decide to modify the stellar luminosity to 0 with 1/3 probability
                if random.random() < 1/3:
                    row['Stellar Luminosity [log(Solar)]'] = '0'
                row['Stellar Mass [Solar mass]'] = '%.3f' % (random.uniform(0.1, 15))
                modified_data.append(row)

            # Fieldnames (i.e., column names) to be used in writing the CSV file
            fieldnames = reader.fieldnames

        # Write the modified data to the same file, overwriting it
        with open(file_path, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()  # Write the column headers
            writer.writerows(modified_data)  # Write the modified data

# Call the required functions as per instructions
random_data(directories[rubric_item])  # Generate completely new randomized data, no size specified, use default
modify_data(directories[rubric_item])  # Modify the generated data based on the test details

In [105]:
rubric_item = 'q6: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q6')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [106]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q6: did not ignore the `Star` objects with missing `stellar_luminosity` data

In [107]:
rubric_item = 'q6: did not ignore the `Star` objects with missing `stellar_luminosity` data'
readme_text = """Check for `None` values when computing average
luminosity. Modify the code to skip stars with
missing `stellar_luminosity` data. Consider using
an `if` condition to filter out such cases."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [108]:
import os
import csv
import json
import random

def modify_data(directory):
    # Define the filenames for stars CSV files
    stars_files = [f'stars_{i}.csv' for i in range(1, 6)]

    for filename in stars_files:
        # Construct the full path to the current stars file
        file_path = os.path.join(directory, 'data', filename)
        # List to store modified rows
        modified_data = []

        # Open the current stars CSV file
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Randomly decide to delete the stellar luminosity with 1/3 probability
                if random.random() < 1/3:
                    row['Stellar Luminosity [log(Solar)]'] = ''
                row['Stellar Mass [Solar mass]'] = '%.3f' % (random.uniform(0.1, 15))
                modified_data.append(row)

            # Fieldnames (i.e., column names) to be used in writing the CSV file
            fieldnames = reader.fieldnames

        # Write the modified data to the same file, overwriting it
        with open(file_path, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()  # Write the column headers
            writer.writerows(modified_data)  # Write the modified data

# Call the required functions as per instructions
random_data(directories[rubric_item])  # Generate completely new randomized data, no size specified, use default
modify_data(directories[rubric_item])  # Modify the generated data based on the test details

In [109]:
rubric_item = 'q6: did not ignore the `Star` objects with missing `stellar_luminosity` data'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q6')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [110]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q6: `star_classes` data structure is not used to answer

In [111]:
rubric_item = 'q6: `star_classes` data structure is not used to answer'
readme_text = """Review the usage of `star_classes` to ensure that
you correctly access the information within the
`star_classes` dictionary, avoiding extra
processing. Additionally, confirm correct
computation of averages, only including stars with
known `stellar_luminosity`."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [112]:
random_data(directories[rubric_item])

In [113]:
rubric_item = 'q6: `star_classes` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_idx = find_all_cell_indices(nb, "code", "grader.check('q6')")[-1]
nb = truncate_nb(nb, end=end_idx)

false_star_classes = true_data_structures["stars_dict"] + "\n" + true_data_structures["star_classes"] + '''
star_classes['Neutron Star'], star_classes['Red Giant'], star_classes['White Dwarf'] = star_classes['White Dwarf'], star_classes['Neutron Star'], star_classes['Red Giant']
'''
nb = replace_with_false_data_structure(nb, 'star_classes', false_star_classes)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [114]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q7: incorrect logic is used to answer

In [115]:
rubric_item = 'q7: incorrect logic is used to answer'
readme_text = """Ensure density and luminosity calculations only
include stars with complete data. Verify your skip
condition and handle cases where mass, radius, or
luminosity may be set to 0 or None. Make sure you
correctly add data to the dictionaries."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [116]:
import os
import random
import csv
import json

def modify_data(directory):
    # Helper function to modify CSV data according to the probability rules
    def modify_csv(file_path):
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            modified_data = []
            for row in reader:
                row["Stellar Mass [Solar mass]"] = '%0.4f' % (random.uniform(0.1, 15))
                # Modify each value with a probability 1/4
                for column in ["Stellar Luminosity [log(Solar)]", "Stellar Radius [Solar Radius]", "Stellar Mass [Solar mass]"]:
                    if random.random() < 0.25:
                        row[column] = ''
                modified_data.append(row)
        
        # Writing the modified data back to the CSV
        with open(file_path, mode='w', encoding='utf-8', newline="") as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(modified_data)
    
    # Iterate over CSV files in the data directory
    for root, dirs, files in os.walk(os.path.join(directory, 'data')):
        for file in files:
            if file.endswith('.csv') and "stars_" in file:
                file_path = os.path.join(root, file)
                modify_csv(file_path)

# Calling functions as per rubric item test details
random_data(directories[rubric_item]) # Call random_data with directories specified in the rubric_item
modify_data(directories[rubric_item])  # Call modify_data with directories specified in the rubric_item

In [117]:
rubric_item = 'q7: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q7')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [118]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q7: `star_classes` data structure is not used to answer

In [119]:
rubric_item = 'q7: `star_classes` data structure is not used to answer'
readme_text = """Ensure you are using the `star_classes` data
structure correctly to access the required
attributes of each `Star` object. Check if you are
iterating through the objects in the right way and
accessing the correct properties. Avoid processing
data that has already been preprocessed. Refer to
the correct data structure as per instructions."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [120]:
rubric_item = 'q7: `star_classes` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q7')")[-1])

# Define the new data structure with swapped star class values
false_star_classes = true_data_structures["stars_dict"] + "\n" + true_data_structures["star_classes"] + '''
star_classes['Neutron Star'], star_classes['Red Giant'], star_classes['White Dwarf'] = star_classes['White Dwarf'], star_classes['Neutron Star'], star_classes['Red Giant']
'''
nb = replace_with_false_data_structure(nb, 'star_classes', false_star_classes)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [121]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q7: plot is incorrect

In [122]:
rubric_item = 'q7: plot is incorrect'
readme_text = """Check that your scatter plot correctly uses
`plot_scatter_multiple` with properly computed
density and luminosity for each star class. Ensure
`None` values are handled appropriately and plot
the data in the expected cell."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [123]:
rubric_item = 'q7: plot is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q7')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 7:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

store_hidn_plot_dict = '''
hidn_plot_dict = None
import matplotlib.pyplot as plt
    
def plot_scatter_multiple(x_data_dict, y_data_dict, x_label='x axis', y_label='y axis'):
    global hidn_plot_dict
    hidn_plot_dict = {
        'x_dict': x_data_dict,
        'y_dict': y_data_dict
    }
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", "**Question 7:**")[-1]+1, store_hidn_plot_dict)

test_code = '''
expected_plots = public_tests.get_expected_plots()
expected_x_dict = expected_plots['q7']['density_dict']
expected_y_dict = expected_plots['q7']['lum_dict']

if hidn_plot_dict:
    for key in expected_x_dict:
        if key in hidn_plot_dict['x_dict'] and public_tests.compare(expected_x_dict[key], hidn_plot_dict['x_dict'][key], 'TEXT_FORMAT_DICT') == public_tests.PASS:
            for ykey in expected_y_dict:
                if ykey in hidn_plot_dict['y_dict'] and public_tests.compare(expected_y_dict[ykey], hidn_plot_dict['y_dict'][ykey], 'TEXT_FORMAT_DICT') == public_tests.PASS:
                    test_output = 'q7 results: ' + public_tests.PASS
'''

inject_code(nb, len(nb['cells']), get_test_text("q7", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
    comments[rubric_item] += '\nFAILED TEST: ' + results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [124]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q7: plot is not properly labeled

In [125]:
rubric_item = "q7: plot is not properly labeled"
readme_text = """The notebook is parsed and the
plot for Question 7 is identified.
The image is parsed, and the text
in the left and bottom of the image
are read.
If any of the required labels are
missing, then the test is deemed
a failure.""" 

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [126]:
rubric_item = "q7: plot is not properly labeled"
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q7')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 7:**")[-1])
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot = get_last_plot(nb, os.path.join(directories[rubric_item], 'plot_image.png'))

if rubric_item not in results:
    results[rubric_item] = {}
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
else:
    left_label_plot = get_label_plot(plot, 'left').rotate(-90)
    left_label_check = check_text_in_plot(left_label_plot, ['Luminosity'])
    bottom_label_plot = get_label_plot(plot, 'bottom')
    bottom_label_check = check_text_in_plot(bottom_label_plot, ['Density'])
    centre_label_plot = get_bounding_box_plot(plot)
    centre_label_check = check_text_in_plot(centre_label_plot, ['Red Giant', 'White Dwarf', 'Neutron Star'])
    missing_labels = left_label_check['missing_text'] + bottom_label_check['missing_text'] + centre_label_check['missing_text']
    extra_labels = left_label_check['extra_text'] + bottom_label_check['extra_text'] + centre_label_check['extra_text']

    if missing_labels == [] and extra_labels == []:
        results[rubric_item] = {rubric_item.split(":")[0]: "All test cases passed!"}
    else:
        if missing_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "missing labels: " + repr(missing_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]
        if extra_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "extra labels: " + repr(extra_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q8: incorrect comparison operator is used

In [127]:
rubric_item = 'q8: incorrect comparison operator is used'
readme_text = """The test checks the comparison operator for
excluding stars with density more than 25. Verify
your comparison operator to ensure it excludes the
correct values. Review your logic for computing
density and filtering by the threshold. Consider
testing with edge cases, such as density exactly
equal to 25."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [128]:
import os
import random
import csv

def modify_data(directory):
    # Define the filenames for star CSV files to be modified
    star_filenames = [f'stars_{i}.csv' for i in range(1, 6)]

    for filename in star_filenames:
        filepath = os.path.join(directory, 'data', filename)
        
        with open(filepath, 'r', encoding='utf-8') as file:
            # Read csv contents
            reader = csv.reader(file)
            # Store the header and data separately
            header = next(reader)
            data = list(reader)
        
        for i in range(len(data)):
            # Update the row
            data[i][4] = '%.3f' % (random.uniform(0.1, 15))
            
        # Calculate the number of rows to alter
        modify_count = len(data) // 2
        rows_to_modify = random.sample(range(len(data)), modify_count)

        for i in rows_to_modify:
            data[i][3] = str(1)
            # Calculate mass as 25 times the cube of the radius
            mass = 25 * float(data[i][3]) ** 3
            # Update the row
            data[i][4] = str(mass)
        
        with open(filepath, 'w', encoding='utf-8', newline='') as file:
            # Write the modified data back to the CSV
            writer = csv.writer(file)
            writer.writerow(header)  # Write the header first
            writer.writerows(data)  # Write the modified data

# Generate random data first
random_data(directories[rubric_item])
# Then call the modify_data function with the directory path
modify_data(directories[rubric_item])

In [129]:
rubric_item = 'q8: incorrect comparison operator is used'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q8')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [130]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q8: `star_classes` data structure is not used to answer

In [131]:
rubric_item = 'q8: `star_classes` data structure is not used to answer'
readme_text = """Check that your code uses the predefined
`star_classes` data structure accurately and
iterates through it correctly. Also, ensure you're
filtering stars based on density as specified and
handling missing data as instructed. Review how
dictionaries are accessed and updated. Adjust your
code accordingly and rerun your tests."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [132]:
rubric_item = 'q8: `star_classes` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q8')")[-1])

false_star_classes = true_data_structures["stars_dict"] + "\n" + true_data_structures["star_classes"] + '''
star_classes['Neutron Star'], star_classes['Red Giant'], star_classes['White Dwarf'] = star_classes['White Dwarf'], star_classes['Neutron Star'], star_classes['Red Giant']
'''
nb = replace_with_false_data_structure(nb, 'star_classes', false_star_classes)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [133]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q8: plot is incorrect

In [134]:
rubric_item = 'q8: plot is incorrect'
readme_text = """Ensure you are using the `plot_scatter_multiple`
function correctly with proper arguments. Check if
you are displaying the plot in the expected cell.
Review dictionary creation and how you filter data
with density over 25. Re-run to see your plot."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [135]:
rubric_item = 'q8: plot is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q8')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 8:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

store_hidn_plot_dict = '''
hidn_plot_dict = None
import matplotlib.pyplot as plt
    
def plot_scatter_multiple(x_data_dict, y_data_dict, x_label='x axis', y_label='y axis'):
    global hidn_plot_dict
    hidn_plot_dict = {
        'x_dict': x_data_dict,
        'y_dict': y_data_dict
    }
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", "**Question 8:**")[-1]+1, store_hidn_plot_dict)

test_code = '''
expected_x_dict = density_dict_no_outliers
expected_y_dict = lum_dict_no_outliers

if hidn_plot_dict:
    for key in expected_x_dict:
        if key in hidn_plot_dict['x_dict'] and public_tests.compare(expected_x_dict[key], hidn_plot_dict['x_dict'][key], 'TEXT_FORMAT_DICT') == public_tests.PASS:
            for ykey in expected_y_dict:
                if ykey in hidn_plot_dict['y_dict'] and public_tests.compare(expected_y_dict[ykey], hidn_plot_dict['y_dict'][ykey], 'TEXT_FORMAT_DICT') == public_tests.PASS:
                    test_output = 'q8 results: ' + public_tests.PASS
'''

inject_code(nb, len(nb['cells']), get_test_text("q8", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
    comments[rubric_item] += '\nFAILED TEST: ' + results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [136]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q8: plot is not properly labeled

In [137]:
rubric_item = "q8: plot is not properly labeled"
readme_text = """The notebook is parsed and the
plot for Question 8 is identified.
The image is parsed, and the text
in the left and bottom of the image
are read.
If any of the required labels are
missing, then the test is deemed
a failure.""" 

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [138]:
rubric_item = "q8: plot is not properly labeled"
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q8')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 8:**")[-1])
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot = get_last_plot(nb, os.path.join(directories[rubric_item], 'plot_image.png'))

if rubric_item not in results:
    results[rubric_item] = {}
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
else:
    left_label_plot = get_label_plot(plot, 'left').rotate(-90)
    left_label_check = check_text_in_plot(left_label_plot, ['Luminosity'])
    bottom_label_plot = get_label_plot(plot, 'bottom')
    bottom_label_check = check_text_in_plot(bottom_label_plot, ['Density'])
    centre_label_plot = get_bounding_box_plot(plot)
    centre_label_check = check_text_in_plot(centre_label_plot, ['Red Giant', 'White Dwarf', 'Neutron Star'])
    missing_labels = left_label_check['missing_text'] + bottom_label_check['missing_text'] + centre_label_check['missing_text']
    extra_labels = left_label_check['extra_text'] + bottom_label_check['extra_text'] + centre_label_check['extra_text']

    if missing_labels == [] and extra_labels == []:
        results[rubric_item] = {rubric_item.split(":")[0]: "All test cases passed!"}
    else:
        if missing_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "missing labels: " + repr(missing_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]
        if extra_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "extra labels: " + repr(extra_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q9: incorrect logic is used to answer

In [139]:
rubric_item = 'q9: incorrect logic is used to answer'
readme_text = """Check that you properly handled cases with missing
data. Verify the filtering logic to make sure that
only stars with all attributes
(`stellar_effective_temperature`,
`stellar_luminosity`, `stellar_age`,
`stellar_mass`) present are included in the lists.
Consider edge cases such as zero values which may
not be intended as missing data."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [140]:
import os
import random
import json
import csv

def modify_data(directory):
    # Helper function to choose if a value should be missing or zeroed with a probability of 1/5
    def random_modification():
        return None if random.random() < 0.2 else 0 if random.random() < 0.2 else True

    # Walk through the 'data' directory to find star CSV files and modify them
    for subfolder in os.listdir(directory):
        if subfolder == 'data':
            data_folder = os.path.join(directory, subfolder)
            for filename in os.listdir(data_folder):
                if filename.startswith('stars') and filename.endswith('.csv'):
                    filepath = os.path.join(data_folder, filename)
                    with open(filepath, 'r', encoding='utf-8') as file:
                        reader = csv.DictReader(file)
                        rows = list(reader)
                    
                    with open(filepath, 'w', encoding='utf-8', newline='') as file:
                        writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                        writer.writeheader()
                        for row in rows:
                            # Randomly modify the rows as needed
                            if random_modification() is None:
                                row['Stellar Luminosity [log(Solar)]'] = None
                            elif random_modification() == 0:
                                row['Stellar Luminosity [log(Solar)]'] = 0

                            if random_modification() is None:
                                row['Stellar Effective Temperature [K]'] = None
                            elif random_modification() == 0:
                                row['Stellar Effective Temperature [K]'] = 0

                            if random_modification() is None:
                                row['Stellar Mass [Solar mass]'] = None
                            elif random_modification() == 0:
                                row['Stellar Mass [Solar mass]'] = 0

                            if random_modification() is None:
                                row['Stellar Age [Gyr]'] = None
                            elif random_modification() == 0:
                                row['Stellar Age [Gyr]'] = 0

                            writer.writerow(row)

# First call random_data to completely modify the dataset
# Please ensure that the random_data function is already defined elsewhere in your code
random_data(directories[rubric_item])

# Now call the modify_data function to apply the specific modifications
modify_data(directories[rubric_item])

In [141]:
rubric_item = 'q9: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q9')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [142]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q9: `stars_dict` data structure is not used to answer

In [143]:
rubric_item = 'q9: `stars_dict` data structure is not used to answer'
readme_text = """Check that you are utilizing the `stars_dict` data
structure correctly to extract and use the
properties of `Star` objects. Your lists should
only include data for `Star` objects that have all
attributes defined. Make sure you are not
reprocessing data outside this structure or
overlooking the requirement to exclude incomplete
data entries."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [144]:
rubric_item = 'q9: `stars_dict` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

end_index = find_all_cell_indices(nb, "code", "grader.check('q9')")[-1]
nb = truncate_nb(nb, end=end_index)

new_stars_dict = true_data_structures['stars_dict'] + '''
import random

random.seed(0)
stars_dict_keys = list(stars_dict.keys())
stars_dict_values = list(stars_dict.values())
random.shuffle(stars_dict_keys)
random.shuffle(stars_dict_values)

stars_dict = {}
for i in range(len(stars_dict_keys)):
    stars_dict[stars_dict_keys[i]] = stars_dict_values[i]
'''

nb = replace_with_false_data_structure(nb, 'stars_dict', new_stars_dict)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [145]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q9: plot is incorrect

In [146]:
rubric_item = 'q9: plot is incorrect'
readme_text = """Ensure you are using the `plot_scatter` function
correctly, passing the right data lists in the
correct order, and that the plot is called in the
expected cell. Check for None values and make sure
to exclude any stars with missing attributes
before plotting."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [147]:
rubric_item = 'q9: plot is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q9')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot_nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 9:**")[-1])
plot = get_last_plot(plot_nb, os.path.join(directories[rubric_item], 'plot_image.png'))

# Inject fake plot_scatter before the question
fake_plot_scatter = '''
hidn_plot_dict = {}
import matplotlib.pyplot as plt    
    
def plot_scatter(x_data, y_data, x_label='x axis', y_label='y axis', c=None, s=7):
    global hidn_plot_dict
    hidn_plot_dict = {
        'x': x_data,
        'y': y_data,
        's': s,
        'c': c
    }
'''
nb = inject_code(nb, find_all_cell_indices(nb, "markdown", '**Question 9:**')[-1]+1, fake_plot_scatter)

# Test code to verify if hidn_plot_dict contains correct data
test_code = '''
expected_hidn_plot_dict = {
    'x': temp_list,
    'y': lum_list,
    's': mass_list,
    'c': age_list,
}

# Comparing each expected element from the plot dictionary with the actual one saved in hidn_plot_dict
comparison_results = []
for key in expected_hidn_plot_dict:
    comparison_results.append(public_tests.compare(expected_hidn_plot_dict[key], hidn_plot_dict.get(key, None), "TEXT_FORMAT"))

if all(result == public_tests.PASS for result in comparison_results):
    test_output = 'q9 results: ' + public_tests.PASS
else:
    test_output = 'q9 results: Not all checks pass.'
'''
nb = inject_code(nb, len(nb['cells']), get_test_text("q9", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
    comments[rubric_item] += '\nFAILED TEST: ' + results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [148]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q9: plot is not properly labeled

In [149]:
rubric_item = "q9: plot is not properly labeled"
readme_text = """The notebook is parsed and the
plot for Question 9 is identified.
The image is parsed, and the text
in the left and bottom of the image
are read.
If any of the required labels are
missing, then the test is deemed
a failure.""" 

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [150]:
rubric_item = "q9: plot is not properly labeled"
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q9')")[-1])
nb = run_nb(nb, os.path.join(directories[rubric_item], FILE))
nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "**Question 9:**")[-1])
if os.path.exists(os.path.join(directories[rubric_item], 'plot_image.png')):
    os.remove(os.path.join(directories[rubric_item], 'plot_image.png'))
plot = get_last_plot(nb, os.path.join(directories[rubric_item], 'plot_image.png'))

if rubric_item not in results:
    results[rubric_item] = {}
if plot == None:
    results[rubric_item][rubric_item.split(":")[0]] = 'plot not found'
else:
    left_label_plot = get_label_plot(plot, 'left').rotate(-90)
    left_label_check = check_text_in_plot(left_label_plot, ['Luminosity'])
    bottom_label_plot = get_label_plot(plot, 'bottom')
    bottom_label_check = check_text_in_plot(bottom_label_plot, ['Effective Temperature'])
    missing_labels = left_label_check['missing_text'] + bottom_label_check['missing_text']
    extra_labels = left_label_check['extra_text'] + bottom_label_check['extra_text']

    if missing_labels == [] and extra_labels == []:
        results[rubric_item] = {rubric_item.split(":")[0]: "All test cases passed!"}
    else:
        if missing_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "missing labels: " + repr(missing_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]
        if extra_labels != []:
            results[rubric_item] = {rubric_item.split(":")[0]: "extra labels: " + repr(extra_labels)}
            comments[rubric_item] += "\n"+results[rubric_item][rubric_item.split(":")[0]]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_all_paths_in: hardcoding the name of directory inside the function

In [151]:
rubric_item = 'get_all_paths_in: hardcoding the name of directory inside the function'
readme_text = """The function failed a test where directories with
names other than "sample_data" were used. Verify
that your function does not include hardcoded
directory names, and ensure it's written to accept
any given directory as an argument. Use the
provided `directory` parameter to build your file
paths."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [152]:
rubric_item = 'get_all_paths_in: hardcoding the name of directory inside the function'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

var_inputs_code = """
var_inputs = [('broken_data',), ('data',)]
"""
nb = inject_function_logic_check(nb, 'get_all_paths_in', var_inputs_code, 'TEXT_FORMAT_UNORDERED_LIST_SLASHES')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [153]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_all_paths_in: function does not remove all files and directories that start with `.`

In [154]:
rubric_item = 'get_all_paths_in: function does not remove all files and directories that start with `.`'
readme_text = """Check that your function correctly ignores files
and directories beginning with a dot (`.`). Review
how you filter such items and ensure they are
excluded from the results before sorting. Apply
recursion properly to handle nested directories."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [155]:
import os
import random

# Define the function to modify data as per the test requirements
def modify_data(directory):
    """
    This function adds new files and directories that start with `.` 
    in all the subdirectories of the specified `broken_data` directory.
    """
    # Helper function to add files or directories
    def add_junk_files_or_dirs(current_dir):
        # List of prefixes to determine type
        prefixes = ['.file_', '.dir_']
        # Randomize the number of junk entries to create
        num_entries = random.randint(1, 5)
        
        for _ in range(num_entries):
            # Randomly choose a name with a prefix
            name = random.choice(prefixes) + str(random.randint(1, 10000))
            path = os.path.join(current_dir, name)
            # Randomly determine whether to create a file or a directory
            if random.choice([True, False]):
                # Create a new file
                with open(path, 'w', encoding='utf-8') as file:
                    file.write('junk data')
            else:
                # Create a new directory
                os.mkdir(path)

    # Recursive function to walk through the directories and create junk files/directories
    def recursive_add_junk(current_dir):
        # First add junk to the current directory
        add_junk_files_or_dirs(current_dir)
        
        # Then iterate over the contents of the current directory
        for entry in os.listdir(current_dir):
            path = os.path.join(current_dir, entry)
            # If entry is a directory (and not a junk file/directory itself), proceed recursively
            if os.path.isdir(path) and not entry.startswith('.'):
                recursive_add_junk(path)
    
    # Start the recursive junk creation process from the initial directory
    recursive_add_junk(os.path.join(directory, 'broken_data'))

# Assuming that 'directories' and 'rubric_item' variables are predefined
# Call the function to modify the dataset, explicitly pass the 'directories[rubric_item]' as argument
random_data(directories[rubric_item])
modify_data(directories[rubric_item])

In [156]:
rubric_item = 'get_all_paths_in: function does not remove all files and directories that start with `.`'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

var_inputs_code = '''
var_inputs = [('broken_data',)]'''
nb = inject_function_logic_check(nb, 'get_all_paths_in', var_inputs_code, test_format="TEXT_FORMAT_UNORDERED_LIST_SLASHES")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [157]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_all_paths_in: function does not sort file names explicitly

In [40]:
rubric_item = 'get_all_paths_in: function does not sort file names explicitly'
readme_text = """Ensure your function explicitly sorts files after
listing them with `os.listdir`, as the order
returned is not guaranteed across different
environments. Check your recursive implementation
for sorting consistency."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [159]:
random_data(directories[rubric_item], 10)

In [41]:
rubric_item = 'get_all_paths_in: function does not sort file names explicitly'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

# Test the function on the variable input "broken_data".
var_inputs_code = '''
var_inputs = [('broken_data',), ('data',)]
'''
nb = inject_function_logic_check(nb, 'get_all_paths_in', var_inputs_code, 'TEXT_FORMAT_ORDERED_LIST_SLASHES')

# Inject code to redefine the string class, and the os functions.
sort_redefine = '''
import os

class newStr(str):
    def __lt__(self, other):
        if isinstance(other, newStr):
            return self > str(other)
        return str(self) < other
    
    def __le__(self, other):
        if isinstance(other, newStr):
            return self >= str(other)
        return str(self) <= other
        
    def __eq__(self, other):
        return str(self) == other

    def __ne__(self, other):
        return str(self) != other

    def __gt__(self, other):
        if isinstance(other, newStr):
            return self < str(other)
        return str(self) > other

    def __ge__(self, other):
        if isinstance(other, newStr):
            return self <= str(other)
        return str(self) >= other
        
    def split(self, sep=None, maxsplit=-1):
        orig_split = str(self).split(sep, maxsplit)
        return [newStr(item) for item in orig_split]

def new_join(*paths):
    return newStr(original_join(*paths))
    
def new_listdir(path):
    return [newStr(p) for p in original_listdir(path)]
    
def new_basename(path):
    return newStr(original_basename(path))
    
def new_dirname(path):
    return newStr(original_dirname(path))

original_join = os.path.join
os.path.join = new_join

original_listdir = os.listdir
os.listdir = new_listdir

original_basename = os.path.basename
os.path.basename = new_basename

original_dirname = os.path.dirname
os.path.dirname = new_dirname
'''

nb = inject_code(nb, 0, sort_redefine)

sort_restore = '''
os.path.join = original_join
os.listdir = original_listdir
os.path.basename = original_basename
os.path.dirname = original_dirname
'''

nb = inject_code(nb, len(nb['cells']), sort_restore)


# Run, parse, and store the outcome of the test in the results dictionary.
results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [42]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_all_paths_in: function logic is incorrect

In [162]:
rubric_item = 'get_all_paths_in: function logic is incorrect'
readme_text = """Ensure your recursive logic correctly explores all
subdirectories and performs the appropriate checks
for file names starting with '.'. Verify that all
paths are relative and that the final list is
sorted in reverse alphabetical order. Use the `os`
module functions appropriately. Consider edge
cases and test with different directory
structures."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [163]:
import os
import random

# Define the modify_data function to test the logic of get_all_paths_in
def modify_data(directory):
    """
    Modify the datasets in the directory according to the modification details. 
    This includes creating new files and directories inside the 'broken_data' directory.
    The contents of the new files and names of the directories are irrelevant as long as they are unique.
    """
    # Define a function to get all the paths in 'broken_data'
    def get_all_paths_in(d):
        paths = []
        for entry in os.listdir(d):
            entry_path = os.path.join(d, entry)
            if os.path.isdir(entry_path):
                paths += get_all_paths_in(entry_path)
            elif os.path.isfile(entry_path) and not entry.startswith('.'):
                paths.append(entry_path)
        return paths

    # Define potential filenames for creation
    new_filenames = ['new_file_1.json', 'new_file_2.json', 'new_file_3.json']

    # Get the current paths in the 'broken_data' directory
    current_paths = get_all_paths_in(os.path.join(directory, 'broken_data'))

    # Determine the path for the 'broken_data' specific directory
    broken_data_path = os.path.join(directory, 'broken_data')

    # Create a random number of directories between 1 and 3 within 'broken_data'
    for i in range(random.randint(1, 3)):
        new_dirname = f'new_dir_{i}'
        new_dirpath = os.path.join(broken_data_path, new_dirname)

        # Ensure the directory does not already exist
        if not os.path.exists(new_dirpath):
            os.mkdir(new_dirpath)

            # Create new files within the new directory
            for new_filename in new_filenames:
                new_filepath = os.path.join(new_dirpath, new_filename)
                with open(new_filepath, 'w', encoding='utf-8') as f:
                    # Write some random content
                    f.write('Random content' + str(random.random()))

# The variables directories and rubric_item have been already defined externally
# If the dataset must be completely modified, call the random_data function
# Since the data must be modified, we call random_data with the project directory
# The size of the data is not specified, so we use the default argument for random_data
random_data(directories[rubric_item])

# Call the modify_data function
modify_data(directories[rubric_item])

In [164]:
rubric_item = 'get_all_paths_in: function logic is incorrect'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

var_inputs_code = '''
var_inputs = [tuple([item[0]]) for item in list(os.walk('broken_data')) if os.path.basename(item[0])[0] != "."]
'''
nb = inject_function_logic_check(nb, 'get_all_paths_in', var_inputs_code, 'TEXT_FORMAT_ORDERED_LIST_SLASHES')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [165]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_all_paths_in: paths are hardcoded using slashes

In [166]:
rubric_item = 'get_all_paths_in: paths are hardcoded using slashes'
readme_text = """Ensure you use `os.path.join` for cross-platform
compatibility instead of hardcoded paths. The test
modifies your code to use a `new_join` function
that concatenates with `&`. Check for proper use
of `os.path.join` in your function."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [167]:
random_data(directories[rubric_item], 100)

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    file_dirs = item[0].split(os.path.sep)
    actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
    remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
    old_path = os.path.join(actual_file_dirs, remaining_path)
    new_path = os.path.join(actual_file_dirs, remaining_path.replace(os.path.sep, '&'))
    if remaining_path == 'broken_data':
        continue
    if os.path.exists(new_path):
        shutil.rmtree(new_path)
    shutil.copytree(old_path, new_path)
    

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    for file in item[2]:
        file_dirs = item[0].split(os.path.sep)
        actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
        remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
        old_path = os.path.join(remaining_path, file)
        new_path = old_path.replace(os.path.sep, '&')
        old_path = os.path.join(actual_file_dirs, old_path)
        new_path = os.path.join(actual_file_dirs, new_path)
        if os.path.exists(new_path):
            os.remove(new_path)
        shutil.copy(old_path, new_path)
        f = open(old_path, 'w', encoding='utf-8')
        f.write('{}')
        f.close()

for file in os.listdir(os.path.join(directories[rubric_item], 'data')):
    if os.path.isfile(os.path.join(directories[rubric_item], 'data', file)):
        shutil.copy(os.path.join(directories[rubric_item], 'data', file), os.path.join(directories[rubric_item], 'data&%s' % (file)))

In [168]:
rubric_item = 'get_all_paths_in: paths are hardcoded using slashes'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

var_inputs_code = '''
var_inputs = [("broken_data",)]
'''
nb = inject_function_logic_check(nb, 'get_all_paths_in', var_inputs_code, 'TEXT_FORMAT_UNORDERED_LIST_SLASHES')

new_join = '''
def new_join(*paths):
    return '&'.join(paths)
    
def new_basename(path):
    return path.split('&')[-1]
    
def new_dirname(path):
    return '&'.join(path.split('&')[:-1])
    
def new_split(path):
    return tuple(['&'.join(path.split('&')[:-1]), path.split('&')[-1]])'''

nb = replace_code(nb, "os.path.join", "new_join")
nb = replace_code(nb, "os.path.basename", "new_basename")
nb = replace_code(nb, "os.path.dirname", "new_dirname")
nb = replace_code(nb, 'os.path.split', 'new_split')
nb = replace_code(nb, 'os.path.sep', "'&'")
nb = inject_code(nb, 0, new_join)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [169]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q10: `get_all_paths_in` function is not used to answer

In [170]:
rubric_item = 'q10: `get_all_paths_in` function is not used to answer'
readme_text = """The test checks if `get_all_paths_in` function is
used properly. Ensure you call this function to
retrieve paths and build them using
`os.path.join`. Review your usage of
`get_all_paths_in` and path construction within
your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [171]:
random_data(directories[rubric_item], 10)

In [172]:
rubric_item = 'q10: `get_all_paths_in` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q10')")[-1])

false_get_all_paths_in = '''
def get_all_paths_in(directory):
    return [directory]
'''
nb = replace_with_false_function(nb, 'get_all_paths_in', false_get_all_paths_in)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [173]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q10: paths are hardcoded using slashes

In [174]:
rubric_item = 'q10: paths are hardcoded using slashes'
readme_text = """Ensure you are using `os.path.join` when
constructing file paths to maintain cross-platform
compatibility. Avoid hardcoding path separators.
Use the provided `get_paths_in` function to list
directory contents. Check the use of
`os.path.join` in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [175]:
random_data(directories[rubric_item], 100)

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    file_dirs = item[0].split(os.path.sep)
    actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
    remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
    old_path = os.path.join(actual_file_dirs, remaining_path)
    new_path = os.path.join(actual_file_dirs, remaining_path.replace(os.path.sep, '&'))
    if remaining_path == 'broken_data':
        continue
    if os.path.exists(new_path):
        shutil.rmtree(new_path)
    shutil.copytree(old_path, new_path)
    f = open(os.path.join(new_path, 'extra_file.json'), 'w')
    f.close()

In [176]:
rubric_item = 'q10: paths are hardcoded using slashes'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q10')")[-1])

code_injection = '''
def new_join(*paths):
    return '&'.join(paths)
    
def new_basename(path):
    return path.split('&')[-1]
    
def new_dirname(path):
    return '&'.join(path.split('&')[:-1])
    
def new_split(path):
    return tuple(['&'.join(path.split('&')[:-1]), path.split('&')[-1]])'''

nb = inject_code(nb, 0, code_injection)
nb = replace_code(nb, "os.path.join", "new_join", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.basename", "new_basename", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.dirname", "new_dirname", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.split', 'new_split', start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.sep', "'&'", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [177]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q11: `get_all_paths_in` function is not used to answer

In [178]:
rubric_item = 'q11: `get_all_paths_in` function is not used to answer'
readme_text = """The test checks if the `get_all_paths_in` function
is used to get paths. Ensure you call this
function with the correct directory path as
argument. Review the use of `os.path.join` and the
function call."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [179]:
random_data(directories[rubric_item], 10)

In [180]:
rubric_item = 'q11: `get_all_paths_in` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q11')")[-1])

new_get_all_paths_in = '''
def get_all_paths_in(directory):
    return [directory]
'''

nb = replace_with_false_function(nb, 'get_all_paths_in', new_get_all_paths_in)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [181]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q11: paths are hardcoded using slashes

In [182]:
rubric_item = 'q11: paths are hardcoded using slashes'
readme_text = """Check that you're using `os.path.join` for
platform-independent path construction. Avoid
hardcoding slashes, and ensure the
`get_all_paths_in` function is called correctly.
Consider different operating systems when
constructing file paths."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [183]:
random_data(directories[rubric_item], 100)

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    file_dirs = item[0].split(os.path.sep)
    actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
    remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
    old_path = os.path.join(actual_file_dirs, remaining_path)
    new_path = os.path.join(actual_file_dirs, remaining_path.replace(os.path.sep, '&'))
    if remaining_path == 'broken_data':
        continue
    if os.path.exists(new_path):
        shutil.rmtree(new_path)
    shutil.copytree(old_path, new_path)
    f = open(os.path.join(new_path, 'extra_file.json'), 'w')
    f.close()

In [184]:
rubric_item = 'q11: paths are hardcoded using slashes'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q11')")[-1])

new_function = '''
def new_join(*paths):
    return '&'.join(paths)
    
def new_basename(path):
    return path.split('&')[-1]
    
def new_dirname(path):
    return '&'.join(path.split('&')[:-1])
    
def new_split(path):
    return tuple(['&'.join(path.split('&')[:-1]), path.split('&')[-1]])'''

nb = inject_code(nb, 0, new_function)
nb = replace_code(nb, "os.path.join", "new_join", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.basename", "new_basename", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.dirname", "new_dirname", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.split', 'new_split', start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.sep', "'&'", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])


results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [185]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q12: `get_all_paths_in` function is not used to answer

In [186]:
rubric_item = 'q12: `get_all_paths_in` function is not used to answer'
readme_text = """The test checks if the `get_all_paths_in` function
is used to obtain file paths. Ensure your solution
calls this function correctly, and that it is not
overridden or modified elsewhere in your code.
Revisit the function's usage and correct your
implementation accordingly."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [187]:
random_data(directories[rubric_item], 10)

In [188]:
rubric_item = 'q12: `get_all_paths_in` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q12')")[-1])

false_get_all_paths_in = '''
def get_all_paths_in(directory):
    return [directory]
'''
nb = replace_with_false_function(nb, 'get_all_paths_in', false_get_all_paths_in)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [189]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q12: paths are hardcoded using slashes

In [190]:
rubric_item = 'q12: paths are hardcoded using slashes'
readme_text = """Feedback: To construct file paths in a
cross-platform manner, utilize `os.path.join()`
instead of hardcoding slashes. This approach
ensures compatibility across different operating
systems. Modify your code to employ
`os.path.join()` for assembling directory paths."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [191]:
random_data(directories[rubric_item], 100)

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    file_dirs = item[0].split(os.path.sep)
    actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
    remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
    old_path = os.path.join(actual_file_dirs, remaining_path)
    new_path = os.path.join(actual_file_dirs, remaining_path.replace(os.path.sep, '&'))
    if remaining_path == 'broken_data':
        continue
    if os.path.exists(new_path):
        shutil.rmtree(new_path)
    shutil.copytree(old_path, new_path)
    f = open(os.path.join(new_path, 'extra_file.json'), 'w')
    f.close()

In [192]:
rubric_item = 'q12: paths are hardcoded using slashes'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q12')")[-1])

inject_at_start_code = '''
def new_join(*paths):
    return '&'.join(paths)
    
def new_basename(path):
    return path.split('&')[-1]
    
def new_dirname(path):
    return '&'.join(path.split('&')[:-1])
    
def new_split(path):
    return tuple(['&'.join(path.split('&')[:-1]), path.split('&')[-1]])'''
nb = inject_code(nb, 0, inject_at_start_code)

nb = replace_code(nb, "os.path.join", "new_join", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.basename", "new_basename", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.dirname", "new_dirname", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.split', 'new_split', start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.sep', "'&'", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [193]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q13: `get_all_paths_in` function is not used to answer

In [194]:
rubric_item = 'q13: `get_all_paths_in` function is not used to answer'
readme_text = """Ensure you are using the `get_all_paths_in`
function to obtain the list of file paths in the
`broken_data` directory. Review your code to
confirm the function's proper usage and that it
returns a list of relative paths as required."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [195]:
random_data(directories[rubric_item], 10)

In [196]:
rubric_item = 'q13: `get_all_paths_in` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q13')")[-1])

false_get_all_paths_in = '''
def get_all_paths_in(directory):
    return [directory]
'''
nb = replace_with_false_function(nb, 'get_all_paths_in', false_get_all_paths_in)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [197]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### all_planets_list: data structure is defined incorrectly

In [198]:
rubric_item = 'all_planets_list: data structure is defined incorrectly'
readme_text = """Please ensure that `all_planets_list` is
initialized as an empty list and that you are
correctly extending it with `planets_list` and the
objects returned by `get_planets`, without
altering `planets_list`. Check your loop and the
use of the `extend` method."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [199]:
rubric_item = 'all_planets_list: data structure is defined incorrectly'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

# Locate the end of the definition of the data structure
end_data_structure = find_all_cell_indices(nb, "code", "grader.check('all_planets_list')")[-1]

# Truncate the notebook to only include cells needed for this test
nb = truncate_nb(nb, end=end_data_structure)

nb = inject_data_structure_check(nb, 'all_planets_list', 'TEXT_FORMAT_UNORDERED_LIST')

# Run the notebook, parse the results, and save the outcome in the variable `results[rubric_item]`
results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [200]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### all_planets_list: `get_planets` function is not used to answer

In [201]:
rubric_item = 'all_planets_list: `get_planets` function is not used to answer'
readme_text = """Check if you are calling `get_planets` as
specified in the question for each file in
`broken_data`, making sure `all_planets_list` is
extended with new Planet objects without altering
`planets_list`. Confirm proper use of file paths
and list methods."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [202]:
random_data(directories[rubric_item], 10)

In [203]:
rubric_item = 'all_planets_list: `get_planets` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('all_planets_list')")[-1])

nb = inject_data_structure_check(nb, 'all_planets_list', "TEXT_FORMAT_UNORDERED_LIST")

new_get_planets = '''
import os
import csv
import json
import random

def process_csv(filename):
    csv_file = open(filename, encoding='utf-8')
    csv_reader = csv.reader(csv_file)
    csv_data = list(csv_reader)
    csv_file.close()
    return csv_data

def read_json(path):
    with open(path, encoding="utf-8") as f:
        return json.load(f)

def get_planets(planet_file, mapping_file):
    if 'data' not in planet_file:
        planet_file = os.path.join('data', planet_file)
    planets = []
    try:
        mapping_dict = read_json(mapping_file)
    except json.JSONDecodeError:
        return []
    planets_csv = process_csv(planet_file)
    planets_header = planets_csv[0]
    planets_rows = planets_csv[1:]
    random.seed(0)
    rows_in_cols = {}
    for i in range(len(planets_rows[0])):
        rows_in_cols[i] = []
        for j in range(len(planets_rows)):
            rows_in_cols[i].append(planets_rows[j][i])
        random.shuffle(rows_in_cols[i])
        
    for j in range(len(planets_rows)):
        for i in range(len(planets_rows[0])):
            planets_rows[j][i] = rows_in_cols[i][j]
            
    mapping_dict_keys = list(mapping_dict.keys())
    mapping_dict_values = list(mapping_dict.values())
    random.shuffle(mapping_dict_keys)
    random.shuffle(mapping_dict_values)
    mapping_dict = {}
    for i in range(len(mapping_dict_keys)):
        mapping_dict[mapping_dict_keys[i]] = mapping_dict_values[i]
            
    for row_idx in range(len(planets_rows)):
        try:
            planet_name = planet_cell(row_idx, 'Planet Name', planets_rows)
            host_name = mapping_dict[planet_name]
            discovery_method = planet_cell(row_idx, 'Discovery Method', planets_rows)
            discovery_year = planet_cell(row_idx, 'Discovery Year', planets_rows)
            controversial_flag = planet_cell(row_idx, 'Controversial Flag', planets_rows)
            orbital_period = planet_cell(row_idx, 'Orbital Period [days]', planets_rows)
            planet_radius = planet_cell(row_idx, 'Planet Radius [Earth Radius]', planets_rows)
            planet_mass = planet_cell(row_idx, 'Planet Mass [Earth Mass]', planets_rows)
            semi_major_radius = planet_cell(row_idx, 'Orbit Semi-Major Axis [au]', planets_rows)
            eccentricity = planet_cell(row_idx, 'Eccentricity', planets_rows)
            equilibrium_temperature = planet_cell(row_idx, 'Equilibrium Temperature [K]', planets_rows)
            insolation_flux = planet_cell(row_idx, 'Insolation Flux [Earth Flux]', planets_rows)
            planet = Planet(planet_name, host_name, discovery_method, discovery_year, controversial_flag, orbital_period, planet_radius, planet_mass, semi_major_radius, eccentricity, equilibrium_temperature, insolation_flux)
            planets.append(planet)
        except IndexError:
            continue
        except ValueError:
            continue
        except KeyError:
            continue
    return planets
'''
nb = replace_with_false_function(nb, 'get_planets', new_get_planets)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [204]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### all_planets_list: `broken_data` data structure is not used to answer

In [205]:
rubric_item = 'all_planets_list: `broken_data` data structure is not used to answer'
readme_text = """The test checks whether `all_planets_list`
correctly includes `Planet` objects from
`planets_list` and the files in `broken_data`.
Ensure your loop iterates over the paths in
`broken_data` and that `get_planets` is called
with the correct arguments. Use `extend` to add to
`all_planets_list`. Avoid altering `planets_list`."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [206]:
import os
import shutil

def modify_data(directory):
    # Define the directories for `broken_data` and its copy `false_broken_data`
    broken_data_dir = os.path.join(directory, 'broken_data')
    false_broken_data_dir = os.path.join(directory, 'false_broken_data')
    if os.path.exists(false_broken_data_dir):
        shutil.rmtree(false_broken_data_dir)
    
    # Copy the `broken_data` directory to `false_broken_data`
    shutil.copytree(broken_data_dir, false_broken_data_dir)
    
    # Delete the original `broken_data` directory
    shutil.rmtree(broken_data_dir)
    
random_data(directories[rubric_item])
modify_data(directories[rubric_item])

In [207]:
rubric_item = 'all_planets_list: `broken_data` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
question_start = find_all_cell_indices(nb, "markdown", "### Data Structure 2: `all_planets_list`")[-1]
question_end = find_all_cell_indices(nb, "code", "grader.check('all_planets_list')")[-1]
nb = truncate_nb(nb, end=question_end)

test_format = "TEXT_FORMAT_UNORDERED_LIST"
nb = inject_data_structure_check(nb, 'all_planets_list', test_format)

code_to_inject = '''
import os

false_broken_data = 'false_broken_data'
broken_data = []

for root, dirs, files in os.walk(false_broken_data):
    for file in files:
        broken_data.append(os.path.join(root, file))
'''
nb = inject_code(nb, question_start + 1, code_to_inject)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### all_planets_list: paths are hardcoded using slashes

In [208]:
rubric_item = 'all_planets_list: paths are hardcoded using slashes'
readme_text = """Review how paths are combined using `os.path.join`
and make sure the paths are not hardcoded. Update
your code to join paths dynamically, considering
differences in operating systems."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [209]:
random_data(directories[rubric_item], 100)

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    file_dirs = item[0].split(os.path.sep)
    actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
    remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
    old_path = os.path.join(actual_file_dirs, remaining_path)
    new_path = os.path.join(actual_file_dirs, remaining_path.replace(os.path.sep, '&'))
    if remaining_path == 'broken_data':
        continue
    if os.path.exists(new_path):
        shutil.rmtree(new_path)
    shutil.copytree(old_path, new_path)
    

for item in list(os.walk(os.path.join(directories[rubric_item], 'broken_data'))):
    for file in item[2]:
        file_dirs = item[0].split(os.path.sep)
        actual_file_dirs = os.path.sep.join(file_dirs[:file_dirs.index('broken_data')])
        remaining_path = os.path.sep.join(file_dirs[file_dirs.index('broken_data'):])
        old_path = os.path.join(remaining_path, file)
        new_path = old_path.replace(os.path.sep, '&')
        old_path = os.path.join(actual_file_dirs, old_path)
        new_path = os.path.join(actual_file_dirs, new_path)
        if os.path.exists(new_path):
            os.remove(new_path)
        shutil.copy(old_path, new_path)
        f = open(old_path, 'w', encoding='utf-8')
        f.write('{}')
        f.close()

for file in os.listdir(os.path.join(directories[rubric_item], 'data')):
    if os.path.isfile(os.path.join(directories[rubric_item], 'data', file)):
        shutil.copy(os.path.join(directories[rubric_item], 'data', file), os.path.join(directories[rubric_item], 'data&%s' % (file)))

In [210]:
rubric_item = 'all_planets_list: paths are hardcoded using slashes'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
question_end = find_all_cell_indices(nb, "code", "grader.check('all_planets_list')")[-1]
nb = truncate_nb(nb, end=question_end)

nb = inject_data_structure_check(nb, 'all_planets_list', "TEXT_FORMAT_UNORDERED_LIST")

new_join = '''
def new_join(*paths):
    return '&'.join(paths)
    
def new_basename(path):
    return path.split('&')[-1]
    
def new_dirname(path):
    return '&'.join(path.split('&')[:-1])
    
def new_split(path):
    return tuple(['&'.join(path.split('&')[:-1]), path.split('&')[-1]])'''
nb = inject_code(nb, 0, new_join)

nb = replace_code(nb, "os.path.join", "new_join", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.basename", "new_basename", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, "os.path.dirname", "new_dirname", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.split', 'new_split', start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])
nb = replace_code(nb, 'os.path.sep', "'&'", start=find_all_cell_indices(nb, "code", "grader.check('get_all_paths_in')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### all_planets_list: `planets_list` data structure is modified or redefined

In [211]:
rubric_item = 'all_planets_list: `planets_list` data structure is modified or redefined'
readme_text = """The test checks if `planets_list` remains
unmodified after your code runs. Ensure that in
your implementation, `planets_list` is not
altered. Copy its contents to `all_planets_list`
without affecting the original `planets_list`. Use
`extend` method carefully to avoid unintended
modifications."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [212]:
rubric_item = 'all_planets_list: `planets_list` data structure is modified or redefined'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

expected_planets_list = true_functions['planet_cell'] + "\n" + true_functions['get_planets'] + "\n"
expected_planets_list += replace_variable(true_data_structures['planets_list'], 'planets_list', 'true_planets_list')

test_code = expected_planets_list + '''
if public_tests.compare(true_planets_list, planets_list, "TEXT_FORMAT_UNORDERED_LIST") == public_tests.PASS:
    test_output = 'all_planets_list results: ' + public_tests.PASS
'''

nb = inject_code(nb, len(nb['cells']), get_test_text("all_planets_list", test_code))

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [213]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_surface_gravity: function did not return `None` for missing data

In [214]:
rubric_item = 'get_surface_gravity: function did not return `None` for missing data'
readme_text = """Check if your function `get_surface_gravity`
returns `None` when either `planet_mass` or
`planet_radius` attribute of the `Planet` object
is `None`. Ensure that missing data is handled
correctly."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [215]:
import os
import random
import csv
import json

# Function to modify datasets according to the rubric being tested.
def modify_data(directory):
    # Loop through all the planets files in the directory.
    planets_files = [f"planets_{i}.csv" for i in range(1, 6)]
    for file_name in planets_files:
        # Construct the full file path using os.path.join to ensure cross-platform compatibility.
        file_path = os.path.join(directory, 'data', file_name)
        
        # Array to hold modified rows.
        modified_rows = []
        
        # Open the file for reading and then writing after modification.
        with open(file_path, mode='r', encoding='utf-8') as file:
            # Read the content of the CSV file.
            csv_reader = csv.DictReader(file)
            # Loop through each row in the CSV file.
            for row in csv_reader:
                # Randomly delete 'Planet Mass [Earth Mass]' or 'Planet Radius [Earth Radius]' with a 1/3 chance each.
                if random.random() < 1/3:
                    row['Planet Mass [Earth Mass]'] = None  # Remove the mass
                if random.random() < 1/3:
                    row['Planet Radius [Earth Radius]'] = None  # Remove the radius
                modified_rows.append(row)  # Add the modified row to the list.
        
        # Rewrite the modified content back to the file.
        with open(file_path, mode='w', encoding='utf-8', newline='') as file:
            # Use the same field names for the DictWriter.
            fields = csv_reader.fieldnames
            csv_writer = csv.DictWriter(file, fieldnames=fields)
            csv_writer.writeheader()  # Write the header to the file.
            csv_writer.writerows(modified_rows)  # Write the modified rows to the file.

# Assuming the `random_data` function, `directories` dictionary, and `rubric_item` are defined externally.
# Generate the completely modified datasets by calling `random_data`.
random_data(directories[rubric_item])

# After generating random data, modify the datasets as per the test details.
modify_data(directories[rubric_item])

In [216]:
rubric_item = 'get_surface_gravity: function did not return `None` for missing data'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "get_surface_gravity")[-1])

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_surface_gravity', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_surface_gravity: function logic is incorrect

In [217]:
rubric_item = 'get_surface_gravity: function logic is incorrect'
readme_text = """Check for proper calculation of gravity ratio
considering that Earth's gravity is a constant
factor. Review the use of physical constants and
ensure that any conditional checks for `None` are
correctly implemented. Consider edge cases and
retest your function with various inputs."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [218]:
rubric_item = 'get_surface_gravity: function logic is incorrect'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_surface_gravity')")[-1])

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_surface_gravity', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q14: `get_surface_gravity` function is not used to answer

In [41]:
rubric_item = 'q14: `get_surface_gravity` function is not used to answer'
readme_text = """Check that you are using the `get_surface_gravity`
function from the module to calculate gravity on
GJ 674 b after locating the planet in the list.
Update your code to use this function if not
already included."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [42]:
rubric_item = 'q14: `get_surface_gravity` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q14')")[-1])

new_get_surface_gravity = '''
def get_surface_gravity(planet):
    # An incorrect version of the function
    mass = planet.planet_mass
    radius = planet.planet_radius
    if mass == None or radius == None:
        return None
    return (mass / radius) * 2 # An incorrect computation
'''
nb = replace_with_false_function(nb, 'get_surface_gravity', new_get_surface_gravity)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [43]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q14: `all_planets_list` data structure is not used to answer

In [44]:
rubric_item = 'q14: `all_planets_list` data structure is not used to answer'
readme_text = """Ensure you are using `all_planets_list` to find
'GJ 674 b'. Iterate over it and compare the
`planet_name` attribute to identify the planet.
After finding the planet, use `break` to exit the
loop. Check your loop and `break` statement
implementation."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [45]:
rubric_item = 'q14: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_index_q14 = find_all_cell_indices(nb, "code", "grader.check('q14')")[-1]
nb = truncate_nb(nb, end=end_index_q14)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
for idx in range(len(all_planets_list)):
    if all_planets_list[idx].planet_name == 'GJ 674 b':
        all_planets_list[idx] = all_planets_list[idx]._replace(planet_mass=10**8, planet_radius=1)
'''
nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [46]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q14: did not exit loop and instead iterated further after finding the answer

In [227]:
rubric_item = 'q14: did not exit loop and instead iterated further after finding the answer'
readme_text = """Ensure that your loop exits immediately after
finding the correct planet. Check if you used
`break` properly to stop iterating through the
list once 'GJ 674 b' is found."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [228]:
import os
import csv
import json
import random

def modify_data(directory):
    # Define the filepaths for planet CSVs and their corresponding mapping JSONs
    planet_files = [f'planets_{i}.csv' for i in range(1, 5)]
    mapping_files = [f'mapping_{i}.json' for i in range(1, 5)]

    for file_index in range(4):
        planets_file_path = os.path.join(directory, 'data', planet_files[file_index])
        mapping_file_path = os.path.join(directory, 'data', mapping_files[file_index])

        # Read and modify the selected planet CSV file
        planets_data = []
        with open(planets_file_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                planets_data.append(row)

        # Pick a random planet from the data and change its name to 'GJ 674 b'
        random_planet = random.choice(planets_data)
        original_planet_name = random_planet['Planet Name']
        random_planet['Planet Name'] = 'GJ 674 b'

        # Write the modified planet data back to the CSV
        with open(planets_file_path, mode='w', encoding='utf-8', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(planets_data)

        # Read the JSON file for mapping
        with open(mapping_file_path, mode='r', encoding='utf-8') as jsonfile:
            mapping_data = json.load(jsonfile)

        # Modify the planet name in the JSON mapping if it exists
        if original_planet_name in mapping_data:
            mapping_data['GJ 674 b'] = mapping_data.pop(original_planet_name)

        # Write the modified mapping back to the JSON file
        with open(mapping_file_path, mode='w', encoding='utf-8') as jsonfile:
            json.dump(mapping_data, jsonfile, ensure_ascii=False, indent=4)

# Call the function random_data with necessary arguments
random_data(directories[rubric_item])  # No size argument given, so default is used

# Call modify_data function with the necessary argument
modify_data(directories[rubric_item])

In [229]:
rubric_item = 'q14: did not exit loop and instead iterated further after finding the answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q14')")[-1])

nb = replace_with_false_data_structure(nb, 'planets_list', true_data_structures['planets_list'])
nb = replace_with_false_data_structure(nb, 'all_planets_list', true_data_structures['all_planets_list'])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [230]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_distances_to_star: function did not return `None` for missing data

In [231]:
rubric_item = 'get_distances_to_star: function did not return `None` for missing data'
readme_text = """Ensure your function checks for `None` in both
`eccentricity` and `semi_major_radius` fields of
the `Planet` object. If either is `None`, the
function should return `None`. Check your
conditionals to correctly handle missing data."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [232]:
import os
import json
import csv
import random

# Define the modify_data function
def modify_data(directory):
    """
    This function modifies the dataset within the 'directory' as specified in the test details.
    For the files planets_1.csv through planets_5.csv, it randomly removes the value from either 
    'Orbit Semi-Major Axis [au]' or 'Eccentricity' column with a probability of 1/3 for each row.
    """
    # Iterate over planet files
    for i in range(1, 6):
        # Construct the planet file path
        planet_file = os.path.join(directory, 'data', f'planets_{i}.csv')
        # Temporarily store modified rows
        modified_rows = []
        
        with open(planet_file, mode='r', encoding='utf-8') as file:
            # Read the file content
            reader = csv.DictReader(file)
            for row in reader:
                # Randomly delete 'Orbit Semi-Major Axis [au]' or 'Eccentricity' value with a probability of 1/3
                if random.random() < 1/3:
                    row['Orbit Semi-Major Axis [au]'] = ''
                if random.random() < 1/3:
                    row['Eccentricity'] = ''
                modified_rows.append(row)
        
        # Write the modified content back to the file
        with open(planet_file, mode='w', encoding='utf-8', newline='') as file:
            # Get fieldnames from the first row of modified_rows
            fieldnames = modified_rows[0].keys()
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            # Write the header
            writer.writeheader()
            # Write the modified rows
            for modified_row in modified_rows:
                writer.writerow(modified_row)

# Modify the entire dataset
random_data(directories[rubric_item])
# Modify the data as per the rubric being tested
modify_data(directories[rubric_item])

In [233]:
rubric_item = 'get_distances_to_star: function did not return `None` for missing data'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_distances_to_star')")[-1])

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_distances_to_star', var_inputs_code, "TEXT_FORMAT_ORDERED_LIST")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_distances_to_star: function logic is incorrect

In [234]:
rubric_item = 'get_distances_to_star: function logic is incorrect'
readme_text = """Check that your function correctly accounts for
the absence of `eccentricity` or
`semi_major_radius` attributes in a `Planet`
object. If either is missing or `None`, the
function should return `None`. Ensure you are
calculating the shortest and longest distances
with the correct formula: `semi_major * (1 -
absolute_value_of_eccentricity)` and `semi_major *
(1 + absolute_value_of_eccentricity)`,
respectively."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [235]:
rubric_item = 'get_distances_to_star: function logic is incorrect'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_distances_to_star')")[-1])

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''

nb = inject_function_logic_check(nb, 'get_distances_to_star', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q15: `get_distances_to_star` function is not used to answer

In [236]:
rubric_item = 'q15: `get_distances_to_star` function is not used to answer'
readme_text = """Ensure you are using the `get_distances_to_star`
function after finding the `planet` object with
the name 'b Cen AB b'. This function is intended
to calculate the distances directly, so make sure
you call this function correctly to get accurate
results."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [237]:
random_data(directories[rubric_item])

In [238]:
rubric_item = 'q15: `get_distances_to_star` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q15')")[-1])

new_get_distances_to_star = '''
def get_distances_to_star(planet):
    ecc = planet.eccentricity
    semi_major = planet.semi_major_radius
    if ecc is None or semi_major is None:
        return None
    return [semi_major / (1 + abs(ecc)), semi_major * (1 + abs(ecc))]
'''
nb = replace_with_false_function(nb, 'get_distances_to_star', new_get_distances_to_star)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [239]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q15: `all_planets_list` data structure is not used to answer

In [240]:
rubric_item = 'q15: `all_planets_list` data structure is not used to answer'
readme_text = """Ensure you're using `all_planets_list` without
altering its structure. If using other techniques
to find the planet, they might not work with the
modified test dataset. Check conditional
statements and potential overwriting of important
variables. Use the predefined data structure to
access the required information efficiently."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [241]:
random_data(directories[rubric_item])

In [242]:
rubric_item = 'q15: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_index = find_all_cell_indices(nb, "code", "grader.check('q15')")[-1]
nb = truncate_nb(nb, end=end_index)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
for idx in range(len(all_planets_list)):
    if all_planets_list[idx].planet_name == 'b Cen AB b':
        all_planets_list[idx] = all_planets_list[idx]._replace(semi_major_radius=10**5, eccentricity=0.5)
'''

nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [243]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q15: did not exit loop and instead iterated further after finding the answer

In [244]:
rubric_item = 'q15: did not exit loop and instead iterated further after finding the answer'
readme_text = """Check the placement of `break` in your loop to
ensure iteration stops after finding 'b Cen AB b'.
Ensure the loop does not continue, which could
lead to incorrect data being processed."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [245]:
import os
import json
import random

def modify_data(directory):
    planets_files = [f'planets_{i}.csv' for i in range(1, 5)]
    mapping_files = [f'mapping_{i}.json' for i in range(1, 5)]
    
    # Randomly choose one of the planet and mapping files
    for index in range(4):  # indices 0 to 3 for files 1 to 4
        chosen_planet_file = planets_files[index]
        chosen_mapping_file = mapping_files[index]

        # Path to the planet file and mapping file
        planet_file_path = os.path.join(directory, 'data', chosen_planet_file)
        mapping_file_path = os.path.join(directory, 'data', chosen_mapping_file)

        # Load planets and make a random modification
        with open(planet_file_path, 'r', encoding='utf-8') as file:
            planets = list(csv.DictReader(file))

        # Pick a random planet to change its name
        random_planet_index = random.randrange(len(planets))
        planets[random_planet_index]['Planet Name'] = 'b Cen AB b'

        # Write the modified data back to the planet file
        with open(planet_file_path, 'w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=planets[0].keys())
            writer.writeheader()
            writer.writerows(planets)

        # Load the mapping file and modify the corresponding planet name
        with open(mapping_file_path, 'r', encoding='utf-8') as file:
            mapping = json.load(file)

        # Change the key corresponding to the modified planet to `b Cen AB b`
        planet_keys = list(mapping.keys())
        mapping['b Cen AB b'] = mapping.pop(planet_keys[random_planet_index])

        # Write the modified data back to the mapping file
        with open(mapping_file_path, 'w', encoding='utf-8') as file:
            json.dump(mapping, file, ensure_ascii=False, indent=4)

random_data(directories[rubric_item])  # The directory for random data generation

# Call modify_data function with the required directory from directories
modify_data(directories[rubric_item])

In [246]:
rubric_item = 'q15: did not exit loop and instead iterated further after finding the answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q15')")[-1])

nb = replace_with_false_data_structure(nb, 'planets_list', true_data_structures['planets_list'])
nb = replace_with_false_data_structure(nb, 'all_planets_list', true_data_structures['all_planets_list'])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [247]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_liquid_water_distances: function did not return `None` for missing data

In [248]:
rubric_item = 'get_liquid_water_distances: function did not return `None` for missing data'
readme_text = """Ensure `get_liquid_water_distances` function
checks for `None` before computing distances.
Review how missing `stellar_luminosity` data is
handled in your function. Revisit the logic for
returning `None` if the data is missing or
incomplete."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [249]:
import os
import random
import pandas as pd

# This function modifies the data as per the provided directions
def modify_data(directory):
    # Iterate over the stars files to modify the relevant data
    for i in range(1, 6):
        # Construct the path to the stars file
        stars_file = os.path.join(directory, f'data', f'stars_{i}.csv')
        
        # Read the stars data using pandas
        stars_df = pd.read_csv(stars_file, encoding='utf-8')
        
        # Modify 'Stellar Luminosity [log(Solar)]' with probability 1/3
        for index, row in stars_df.iterrows():
            if random.random() < 1/3:
                stars_df.at[index, 'Stellar Luminosity [log(Solar)]'] = None
        
        # Write the modified data back to the file
        stars_df.to_csv(stars_file, index=False, encoding='utf-8')

# Assuming 'directories' and 'rubric_item' are predefined
# The dataset must be completely modified
random_data(directories[rubric_item])  # Call random_data without specifying the size, using default argument

# Call modify_data afterwards
modify_data(directories[rubric_item])

In [250]:
rubric_item = 'get_liquid_water_distances: function did not return `None` for missing data'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

function_qnum = "get_liquid_water_distances"
end_of_function = find_all_cell_indices(nb, "code", "grader.check('{}')".format(function_qnum))[-1]
nb = truncate_nb(nb, end=end_of_function)

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, function_qnum, var_inputs_code, "TEXT_FORMAT_ORDERED_LIST")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_liquid_water_distances: function logic is incorrect

In [251]:
rubric_item = 'get_liquid_water_distances: function logic is incorrect'
readme_text = """Check the function logic against varying inputs
and ensure handling of `None` for
`stellar_luminosity`. Consider edge cases and
verify the formula used for distance calculation.
Make sure to retrieve `stellar_luminosity`
correctly from `stars_dict` using
`planet.host_name`."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [252]:
rubric_item = 'get_liquid_water_distances: function logic is incorrect'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
function_qnum = "get_liquid_water_distances"
end_of_function = find_all_cell_indices(nb, "code", "grader.check('{}')".format(function_qnum))[-1]
nb = truncate_nb(nb, end=end_of_function)

# Since we are testing the logic of a particular function, we won't be truncating the notebook.
var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_liquid_water_distances', var_inputs_code, "TEXT_FORMAT_ORDERED_LIST")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q16: `get_liquid_water_distances` function is not used to answer

In [253]:
rubric_item = 'q16: `get_liquid_water_distances` function is not used to answer'
readme_text = """Ensure you use the `get_liquid_water_distances`
function on the identified planet to obtain the
correct distances. This function is designed to
give the correct result efficiently. Double-check
you're calling the function after locating the
planet with the specified name."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [254]:
random_data(directories[rubric_item]) # size is not mentioned, hence omitted

In [255]:
rubric_item = 'q16: `get_liquid_water_distances` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q16')")[-1])

new_get_liquid_water_distances = '''
def get_liquid_water_distances(planet):
    lum = stars_dict[planet.host_name].stellar_luminosity
    if lum == None:
        return None
    return [(10 ** (lum - 1) / 1.15) ** 0.5, (10 ** (lum + 1) / 0.53) ** 0.5]
'''
nb = replace_with_false_function(nb, 'get_liquid_water_distances', new_get_liquid_water_distances)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [256]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q16: `all_planets_list` data structure is not used to answer

In [257]:
rubric_item = 'q16: `all_planets_list` data structure is not used to answer'
readme_text = """Review the use of `all_planets_list` to ensure you
are correctly identifying 'Kepler-197 e' using a
loop and breaking out of it after finding the
planet, as demonstrated in the solution. Use
provided data structures to avoid unnecessary
reprocessing."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [258]:
random_data(directories[rubric_item])

In [259]:
rubric_item = 'q16: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

q16_end_index = find_all_cell_indices(nb, "code", "grader.check('q16')")[-1]
nb = truncate_nb(nb, end=q16_end_index)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
import random
kepler_idx = None
for idx in range(len(all_planets_list)):
    if all_planets_list[idx].planet_name == 'Kepler-197 e':
        kepler_idx = idx
        break

new_hosts = set()
for idx in range(len(all_planets_list)):
    if all_planets_list[idx].host_name not in [None, all_planets_list[kepler_idx].host_name]:
        new_hosts.add(all_planets_list[idx].host_name)  

random.seed(0)
new_host = random.choice(sorted(new_hosts))
all_planets_list[kepler_idx] = all_planets_list[kepler_idx]._replace(host_name=new_host)        
'''
nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [260]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q16: did not exit loop and instead iterated further after finding the answer

In [261]:
rubric_item = 'q16: did not exit loop and instead iterated further after finding the answer'
readme_text = """Ensure you break out of the loop after finding
'Kepler-197 e'. Check if your loop continues to
iterate through all planets even after finding the
desired one, which is not necessary and less
efficient."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [262]:
import os
import json
import csv
import random

def modify_data(directory):
    # Helper function to generate a random planet name
    
    # List of planets_*.csv and mapping_*.json files
    planet_files = [f"planets_{i}.csv" for i in range(1, 5)]
    mapping_files = [f"mapping_{i}.json" for i in range(1, 5)]
    
    # Iterate over planet files to modify one planet name in each
    for i in range(len(planet_files)):
        # Define full path to planet file using os.path.join
        full_planets_path = os.path.join(directory, "data", planet_files[i])
        full_mapping_path = os.path.join(directory, "data", mapping_files[i])
        
        # Create a temporary list to hold modified rows
        with open(full_planets_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            chosen_row = copy.deepcopy(random.choice([row for row in reader])) # Randomly select a row to change
            chosen_planet_name = chosen_row['Planet Name']
            # Edit the chosen row's planet name
            chosen_row['Planet Name'] = 'Kepler-197 e'
            # Start from the beginning after choice
            csvfile.seek(0)
            # Process all rows and apply modification
            new_rows = [chosen_row if row['Planet Name'] == chosen_planet_name else row for row in reader]

        # Write the modified rows back to the CSV file
        with open(full_planets_path, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(new_rows)

        # Read and modify the selected mapping JSON file
        with open(full_mapping_path, 'r', encoding='utf-8') as jsonfile:
            mapping_data = json.load(jsonfile)
            # Change the planet name in the mapping
            if chosen_planet_name in mapping_data:
                mapping_data['Kepler-197 e'] = mapping_data.pop(chosen_planet_name)
        with open(full_mapping_path, 'w', encoding='utf-8') as jsonfile:
            json.dump(mapping_data, jsonfile, indent=2)

random_data(directories[rubric_item]) # Use default argument if size is not specified
modify_data(directories[rubric_item])

In [263]:
rubric_item = 'q16: did not exit loop and instead iterated further after finding the answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q16')")[-1])

nb = replace_with_false_data_structure(nb, 'planets_list', true_data_structures['planets_list'])
nb = replace_with_false_data_structure(nb, 'all_planets_list', true_data_structures['all_planets_list'])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [264]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q17: incorrect logic is used to answer

In [265]:
rubric_item = 'q17: incorrect logic is used to answer'
readme_text = """Ensure your code checks for missing data as
described in the question and applies correct
logic to compare actual and safe distances. Review
use of conditional statements and data validation
in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [266]:
import os
import random
import csv
import json

def modify_data(directory):
    # Helper function to randomly delete a value with a given probability
    def delete_value(probability):
        return None if random.random() < probability else ''

    # Define the probability values
    ECC_SEMIMAJOR_DELETE_PROB = 0.25
    STELLAR_LUMINOSITY_DELETE_PROB = 0.25

    # Iterate over the files planets_1.csv to planets_4.csv
    for i in range(1, 5):
        planets_filename = f'planets_{i}.csv'
        path_to_planets = os.path.join(directory, 'data', planets_filename)

        with open(path_to_planets, mode='r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            rows = list(reader)

        # Modify some rows with regards to Eccentricity and Orbit Semi-Major Axis
        for row in rows[1:]:
            if delete_value(ECC_SEMIMAJOR_DELETE_PROB):
                row[8] = ''  # Eccentricity index
            if delete_value(ECC_SEMIMAJOR_DELETE_PROB):
                row[7] = ''  # Orbit Semi-Major Axis index

        with open(path_to_planets, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)

    # Iterate over the files stars_1.csv to stars_4.csv
    for i in range(1, 5):
        stars_filename = f'stars_{i}.csv'
        path_to_stars = os.path.join(directory, 'data', stars_filename)

        with open(path_to_stars, mode='r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            rows = list(reader)

        # Modify some rows with regards to Stellar Luminosity
        for row in rows[1:]:
            if delete_value(STELLAR_LUMINOSITY_DELETE_PROB):
                row[5] = ''  # Stellar Luminosity index

        with open(path_to_stars, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(rows)

# Call random_data to generate a completely new dataset
random_data(directories[rubric_item])

# Call modify_data function with the updated directory path
modify_data(directories[rubric_item])

In [267]:
rubric_item = 'q17: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q17')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [268]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q17: `get_liquid_water_distances` function is not used to answer

In [269]:
rubric_item = 'q17: `get_liquid_water_distances` function is not used to answer'
readme_text = """The test checks if `get_liquid_water_distances`
function is used correctly. Make sure you are
utilizing this function to compare the planets'
actual distances with the range where liquid water
can exist. Review the function calls and the logic
that compares the distance ranges."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [270]:
random_data(directories[rubric_item], 100)

In [271]:
rubric_item = 'q17: `get_liquid_water_distances` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q17')")[-1])

false_get_liquid_water_distances = '''
def get_liquid_water_distances(planet):
    lum = stars_dict[planet.host_name].stellar_luminosity
    if lum == None:
        return None
    return [(10 ** (lum - 1) / 1.15) ** 0.5, (10 ** (lum + 1) / 0.53) ** 0.5]
'''
nb = replace_with_false_function(nb, 'get_liquid_water_distances', false_get_liquid_water_distances)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [272]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q17: `all_planets_list` data structure is not used to answer

In [273]:
rubric_item = 'q17: `all_planets_list` data structure is not used to answer'
readme_text = """Check if you are correctly iterating over
`all_planets_list` to access each `Planet`
object's data. Ensure you use the provided
`get_distances_to_star` and
`get_liquid_water_distances` functions
appropriately with each `Planet` object. Avoid
redefining or not using `all_planets_list`."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [274]:
random_data(directories[rubric_item], 100)

In [275]:
rubric_item = 'q17: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
question_end_idx = find_all_cell_indices(nb, "code", "grader.check('q17')")[-1]
nb = truncate_nb(nb, end=question_end_idx)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in all_planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
all_planets_list = [Planet(*planet) for planet in raw_planets]
'''

nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [276]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_surface_temperatures: function did not return `None` for missing data

In [277]:
rubric_item = 'get_surface_temperatures: function did not return `None` for missing data'
readme_text = """Check the handling of missing
'equilibrium_temperature' data in your
`get_surface_temperatures` function. Ensure it
returns `None` when this data is not present.
Revisit your condition to detect missing data and
provide the correct response."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [278]:
import os
import random
import csv

def modify_data(directory):
    # Iterate over the planet files that need to be modified
    for i in range(1, 5):
        planet_file_name = f'planets_{i}.csv'
        planet_file_path = os.path.join(directory, 'data', planet_file_name)

        # Initialize a list to store modified rows for each planet file
        modified_rows = []

        with open(planet_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            # Iterate over rows and modify the 'Equilibrium Temperature [K]'
            # column with a probability of 1/3 (33% chance)
            for row in reader:
                if random.random() < 1/3:
                    row['Equilibrium Temperature [K]'] = ''
                modified_rows.append(row)

        # Write the modified data back to the planet file
        with open(planet_file_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()
            writer.writerows(modified_rows)

# First completely modify dataset
random_data(directories[rubric_item])

# Then proceed to modify it as per the modification details
modify_data(directories[rubric_item])

In [279]:
rubric_item = 'get_surface_temperatures: function did not return `None` for missing data'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_surface_temperatures')")[-1])

# Inject code at the end of the notebook to create a Planet object with missing data and check the function's behavior
var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_surface_temperatures', var_inputs_code, "TEXT_FORMAT_ORDERED_LIST")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_surface_temperatures: function logic is incorrect

In [280]:
rubric_item = 'get_surface_temperatures: function logic is incorrect'
readme_text = """Check your function's logic, especially how it's
computing temperatures with different albedo
values. Verify that you are following the correct
mathematical formula and returning a list of
floats. Ensure you handle `None` for missing
`equilibrium_temperature` appropriately."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [281]:
rubric_item = 'get_surface_temperatures: function logic is incorrect'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_surface_temperatures')")[-1])

var_inputs_code = '''
var_inputs = [(planet,) for planet in all_planets_list]
'''
nb = inject_function_logic_check(nb, 'get_surface_temperatures', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q18: `get_surface_temperatures` function is not used to answer

In [282]:
rubric_item = 'q18: `get_surface_temperatures` function is not used to answer'
readme_text = """Ensure the solution utilizes the
`get_surface_temperatures` function after
identifying the planet. Review the usage of this
function for correctness and verify it's not
altered or redefined incorrectly in the notebook."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [283]:
random_data(directories[rubric_item])

In [284]:
rubric_item = 'q18: `get_surface_temperatures` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q18')")[-1])

new_get_surface_temperatures = '''
def get_surface_temperatures(planet):
    return [planet.equilibrium_temperature, 2 * planet.equilibrium_temperature]
'''
nb = replace_with_false_function(nb, 'get_surface_temperatures', new_get_surface_temperatures)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [285]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q18: `all_planets_list` data structure is not used to answer

In [286]:
rubric_item = 'q18: `all_planets_list` data structure is not used to answer'
readme_text = """Ensure you are using the `all_planets_list` data
structure to find the planet 'HD 20794 d'. Check
that you compare the planet name correctly and use
a list of floats format for the result. Consider
edge cases where the planet might not be found."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [287]:
random_data(directories[rubric_item])

In [288]:
rubric_item = 'q18: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

# find the index where Question 18 ends
q18_end_idx = find_all_cell_indices(nb, "code", "grader.check('q18')")[-1]
nb = truncate_nb(nb, end=q18_end_idx)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
for idx in range(len(all_planets_list)):
    if all_planets_list[idx].planet_name == 'HD 20794 d':
        all_planets_list[idx] = all_planets_list[idx]._replace(equilibrium_temperature=10**6)
'''
nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [289]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q18: did not exit loop and instead iterated further after finding the answer

In [290]:
rubric_item = 'q18: did not exit loop and instead iterated further after finding the answer'
readme_text = """Check if you are breaking the loop after finding
'HD 20794 d'. The code should stop iterating once
the correct planet is found to avoid unnecessary
computations. Confirm your loop contains a `break`
statement in the right place."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [291]:
import os
import random
import json
import csv

def modify_data(directory):
    """Modify the dataset as per the instructions to not use the `all_planets_list` data structure."""
    # Choose a random planets file and corresponding mapping file
    for rand_num in range(1, 5):
        planets_file_name = f'planets_{rand_num}.csv'
        mapping_file_name = f'mapping_{rand_num}.json'

        # Generate the paths for the chosen files
        planets_file_path = os.path.join(directory, 'data', planets_file_name)
        mapping_file_path = os.path.join(directory, 'data', mapping_file_name)

        # Read the planet data and choose a random planet to modify
        with open(planets_file_path, mode='r', encoding='utf-8') as planets_file:
            planets_data = list(csv.DictReader(planets_file))
            random_planet_index = random.randint(0, len(planets_data) - 1)

        # Modify the chosen planet
        target_planet_name = planets_data[random_planet_index]['Planet Name']
        planets_data[random_planet_index]['Planet Name'] = 'HD 20794 d'

        # Write the modified planet data back to the file
        with open(planets_file_path, mode='w', encoding='utf-8', newline='') as planets_file:
            writer = csv.DictWriter(planets_file, fieldnames=planets_data[0].keys())
            writer.writeheader()
            writer.writerows(planets_data)

        # Modify the mapping json file
        with open(mapping_file_path, mode='r', encoding='utf-8') as mapping_file:
            mapping_data = json.load(mapping_file)
            # Replace the planet name in the mapping
            if target_planet_name in mapping_data:
                star_name = mapping_data[target_planet_name]
                del mapping_data[target_planet_name]
                mapping_data['HD 20794 d'] = star_name

        # Write the modified mapping data back to the file
        with open(mapping_file_path, mode='w', encoding='utf-8') as mapping_file:
            json.dump(mapping_data, mapping_file, ensure_ascii=False, indent=4)

# Call the random_data function for complete data modification
random_data(directories[rubric_item])

# Call the modify_data function to introduce the modifications that are needed for the test
modify_data(directories[rubric_item])

In [292]:
rubric_item = 'q18: did not exit loop and instead iterated further after finding the answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q18')")[-1])

nb = replace_with_false_data_structure(nb, 'planets_list', true_data_structures['planets_list'])
nb = replace_with_false_data_structure(nb, 'all_planets_list', true_data_structures['all_planets_list'])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [293]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q19: incorrect comparison operators are used

In [294]:
rubric_item = 'q19: incorrect comparison operators are used'
readme_text = """Ensure you use the correct comparison operators to
check if the temperature is strictly greater than
263 and strictly less than 323. Check boundary
conditions and how you handle `None` values.
Consider revising your comparisons and retesting
with varying data."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [295]:
import os
import json
import csv
import random

def modify_data(directory):
    # Base names for planets files
    planets_files = [f"planets_{i}.csv" for i in range(1, 5)]

    # Modify planets data files
    for filename in planets_files:
        filepath = os.path.join(directory, "data", filename)
        with open(filepath, "r", encoding='utf-8') as file:
            reader = csv.DictReader(file)
            rows = list(reader)

        # Modify 'Equilibrium Temperature [K]' column for some rows
        for row in rows:
            # Random modification with given probabilities
            prob = random.random()  # Generate a random number between 0 and 1
            if prob < 1/3:
                row['Equilibrium Temperature [K]'] = '323'
            elif prob < 2/3:
                row['Equilibrium Temperature [K]'] = str(263/((0.5)**4))

        # Write the modified data back to the file
        with open(filepath, "w", encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()  # Write the header to the file
            writer.writerows(rows)  # Write modified rows to the file

# Generate random data
# The random_data function is already defined, do not redefine it
random_data(directories[rubric_item])

# Call the modify_data function with the directory path
modify_data(directories[rubric_item])

In [296]:
rubric_item = 'q19: incorrect comparison operators are used'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q19')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [297]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q19: `get_surface_temperatures` function is not used to answer

In [298]:
rubric_item = 'q19: `get_surface_temperatures` function is not used to answer'
readme_text = """Check if you are using the
`get_surface_temperatures` function correctly to
filter `Planet` objects. Utilize this function to
obtain temperature data, and apply the specified
conditions to determine the planet names to
include in your list."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [299]:
random_data(directories[rubric_item], 100)

In [300]:
rubric_item = 'q19: `get_surface_temperatures` function is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q19')")[-1])

new_get_surface_temperatures = '''
def get_surface_temperatures(planet):
    return [278.15, 314.15]  # fixed temperatures that do not depend on planet data
'''
nb = replace_with_false_function(nb, 'get_surface_temperatures', new_get_surface_temperatures)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [301]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q19: `all_planets_list` data structure is not used to answer

In [302]:
rubric_item = 'q19: `all_planets_list` data structure is not used to answer'
readme_text = """Ensure your code is using the `all_planets_list`
as provided, and correctly filters planets based
on their minimum and maximum surface temperatures.
Recheck comparison operators and logical
conjunctions you've used to enforce the specified
temperature range."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [303]:
random_data(directories[rubric_item], 100)

In [304]:
rubric_item = 'q19: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in all_planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
all_planets_list = [Planet(*planet) for planet in raw_planets]
'''

nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

# Place to store the modified and executed notebook's outcomes.
results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [305]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q20: incorrect logic is used to answer

In [306]:
rubric_item = 'q20: incorrect logic is used to answer'
readme_text = """Check your logic for comparing gravitational pull,
distance range for liquid water, and surface
temperatures. Ensure none of these values are
`None` before comparisons and that your
comparisons accurately reflect the question's
criteria."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [307]:
import os
import pandas as pd
import numpy as np

def modify_data(directory):
    # Modify data in planets files
    for i in range(1, 5):  # Iterating through planets_1.csv to planets_4.csv
        filename = f"planets_{i}.csv"
        file_path = os.path.join(directory, "data", filename)
        
        # Read the planets data
        planets_data = pd.read_csv(file_path, encoding='utf-8')
        # For columns other than 'Planet Name', delete the value with probability 1/20
        for col in planets_data.columns:
            if col != "Planet Name":
                # Apply a lambda function to potentially replace each entry with np.nan
                planets_data[col] = planets_data[col].apply(lambda x: np.nan if np.random.rand() < 1/20 else x)
        
        # Write the modified data back to the csv
        planets_data.to_csv(file_path, index=False, encoding='utf-8')
    
    # Modify data in stars files
    for i in range(1, 5):  # Iterating through stars_1.csv to stars_4.csv
        filename = f"stars_{i}.csv"
        file_path = os.path.join(directory, "data", filename)
        
        # Read the stars data
        stars_data = pd.read_csv(file_path, encoding='utf-8')
        # For columns other than 'Star Name', delete the value with probability 1/20
        for col in stars_data.columns:
            if col != "Star Name":
                # Apply a lambda function to potentially replace each entry with np.nan
                stars_data[col] = stars_data[col].apply(lambda x: np.nan if np.random.rand() < 1/20 else x)
        
        # Write the modified data back to the csv
        stars_data.to_csv(file_path, index=False, encoding='utf-8')

# Modify the data as required by the rubric item being tested
# Directories and rubric item variable already defined earlier
random_data(directories[rubric_item])
modify_data(directories[rubric_item])

In [308]:
rubric_item = 'q20: incorrect logic is used to answer'
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q20')")[-1])

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [309]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q20: `get_surface_gravity`, `get_distances_to_star`, `get_liquid_water_distances`, and `get_surface_temperatures` functions are not used to answer

In [310]:
rubric_item = 'q20: `get_surface_gravity`, `get_distances_to_star`, `get_liquid_water_distances`, and `get_surface_temperatures` functions are not used to answer'
readme_text = """Ensure you are using `get_surface_gravity`,
`get_distances_to_star`,
`get_liquid_water_distances`, and
`get_surface_temperatures` functions correctly to
compute the criteria for habitable planets. If
these functions are not called or implemented as
expected, check function definitions and their
usage in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [311]:
random_data(directories[rubric_item])

In [312]:
rubric_item = 'q20: `get_surface_gravity`, `get_distances_to_star`, `get_liquid_water_distances`, and `get_surface_temperatures` functions are not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
end_idx = find_all_cell_indices(nb, "code", "grader.check('q20')")[-1]
nb = truncate_nb(nb, end=end_idx)

new_get_surface_gravity = '''
def get_surface_gravity(planet):
    return 1.0  # A constant value which is inside the required range
'''

new_get_distances_to_star = '''
def get_distances_to_star(planet):
    return [3, 5]  # Overriding with some constant values
'''

new_get_liquid_water_distances = '''
def get_liquid_water_distances(planet):
    return [1, 10]  # Overriding with some constant values that contain the required range
'''

new_get_surface_temperatures = '''
def get_surface_temperatures(planet):
    return [250, 300]  # Overriding with some constant temperatures that meet the criteria
'''

nb = replace_with_false_function(nb, 'get_surface_gravity', new_get_surface_gravity)
nb = replace_with_false_function(nb, 'get_distances_to_star', new_get_distances_to_star)
nb = replace_with_false_function(nb, 'get_liquid_water_distances', new_get_liquid_water_distances)
nb = replace_with_false_function(nb, 'get_surface_temperatures', new_get_surface_temperatures)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [313]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q20: `all_planets_list` data structure is not used to answer

In [314]:
rubric_item = 'q20: `all_planets_list` data structure is not used to answer'
readme_text = """Check if you are using `all_planets_list`
correctly. Ensure you are iterating over it and
extracting planetary data properly. If you define
and use another data structure, make sure to
remove it and rely on `all_planets_list`. Review
the solution for proper use of this predefined
data structure."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [315]:
random_data(directories[rubric_item])

In [316]:
rubric_item = 'q20: `all_planets_list` data structure is not used to answer'
nb = read_nb(os.path.join(DIRECTORY, FILE))
nb = new_clean_nb(nb)
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q20')")[-1])

new_all_planets_list = true_data_structures['planets_list'] + "\n" + true_data_structures['all_planets_list'] + '''
import random

random.seed(0)
raw_planets = [list(planet) for planet in all_planets_list]
rows_in_cols = {}
for i in range(len(raw_planets[0])):
    rows_in_cols[i] = []
    for j in range(len(raw_planets)):
        rows_in_cols[i].append(raw_planets[j][i])
    random.shuffle(rows_in_cols[i])

for j in range(len(raw_planets)):
    for i in range(len(raw_planets[0])):
        raw_planets[j][i] = rows_in_cols[i][j]
all_planets_list = [Planet(*planet) for planet in raw_planets]
'''

nb = replace_with_false_data_structure(nb, 'all_planets_list', new_all_planets_list)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [317]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### general_deductions: Outputs not visible/did not save the notebook file prior to running the cell containing "export". We cannot see your output if you do not save before generating the zip file.

In [318]:
rubric_item = "general_deductions: Outputs not visible/did not save the notebook file prior to running the cell containing \"export\". We cannot see your output if you do not save before generating the zip file."
nb = new_clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = rubric_item.split(":")[1].strip()
if detect_restart_and_run_all(nb):
    results[rubric_item]['general_deductions'] = "All test cases passed!"

### general_deductions: Used concepts/modules such as csv.DictReader, os.walk, and pandas not covered in class yet. Note that built-in functions that you have been introduced to can be used.

In [319]:
rubric_item = "general_deductions: Used concepts/modules such as csv.DictReader, os.walk, and pandas not covered in class yet. Note that built-in functions that you have been introduced to can be used."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

function_calls = []
for cell in nb['cells']:
    if cell['cell_type'] != "code":
        continue
    for node in ast.walk(ast.parse(cell['source'])):
        if isinstance(node, ast.Call):
            function_calls.append(ast.unparse(node.func))
            
bad_function_calls = False
for bad_function in function_calls:
    if 'DictReader' in bad_function or 'pandas' in bad_function or ('os' in bad_function and 'walk' in bad_function):
        bad_function_calls = True
        break

results[rubric_item] = {}
found_imports = set(detect_imports(nb)) - {"otter", "public_tests", "copy", "csv", "json", "json.JSONDecodeError", 
                                           "os", "collections.namedtuple", "collections", "math", "statistics", 
                                           "matplotlib.pyplot", "matplotlib"}
if found_imports  == set():
    if bad_function_calls == False:
        results[rubric_item]['general_deductions'] = "All test cases passed!"
    else:
        results[rubric_item]['general_deductions'] = "found unexpected function call:\n" + repr(bad_function)
        comments[rubric_item] = results[rubric_item]['general_deductions']
else:
    results[rubric_item]['general_deductions'] = "found unexpected import(s):" + repr(list(found_imports))
    comments[rubric_item] = results[rubric_item]['general_deductions']

### general_deductions: Large outputs such as stars_dict or planets_list are displayed in the notebook.

In [320]:
rubric_item = "general_deductions: Large outputs such as stars_dict or planets_list are displayed in the notebook."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = 'All test cases passed!'

for cell in nb['cells']:
    if cell['cell_type'] != "code":
        continue
    output = ""
    if 'outputs' not in cell:
        continue
    for output_cell in cell['outputs']:
        if 'text' in output_cell:
            output += output_cell["text"]+"\n"
        elif 'data' in output_cell and 'text/plain' in output_cell['data']:
            output += output_cell["data"]["text/plain"] + "\n"
    if len(output) > 10**6:
        results[rubric_item]['general_deductions'] = "large outputs detected in notebook"
        break

### general_deductions: Import statements are not mentioned in the required cell at the top of the notebook.

In [321]:
rubric_item = "general_deductions: Import statements are not mentioned in the required cell at the top of the notebook."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "### Loading in the Stars and Planets:")[0]+1, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = 'All test cases passed!'

found_imports = detect_imports(nb)
if found_imports != []:
    results[rubric_item]['general_deductions'] = "found unexpected import(s):" + repr(found_imports)
    comments[rubric_item] = results[rubric_item]['general_deductions']